# Task 2: Training Models

## Data Cleaning of the Datasets

Mount the google drive folder

In [52]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

### Dataset cleaning process

The aim of this section is to create a single dataset using the two datasets at our disposal to simplify the retrieval of the information. Specifically we are interested in the descriptions of the problems, the various examples for the solutions and the respective time and space complexities. The aim is to simplify the retrieval of the information contained in the datasets. We then proceed to eliminate the na values, fix the index, and eliminate the comments from the code solutions.


First, the datasets problem_and_human_solutions_list and complexity_labels_light are downloaded and turned into DataFrames.




In [53]:
from huggingface_hub import hf_hub_download
import json
import pandas as pd
import numpy as np

In [54]:
# Download the dataset from Hugging Face Hub
file_path_1 = hf_hub_download(
    repo_id="facebook/BigOBench",
    filename="data/problem_and_human_solutions_list.jsonl",
    repo_type="dataset"
)

# Read the JSONL file and convert it to a DataFrame
df_problem_and_human_solutions_list = []
with open(file_path_1, "r", encoding="utf-8") as f:
    for line in f:
        try:
            df_problem_and_human_solutions_list.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("Erorr JSON:", e)

df_problem_and_human_solutions_list = pd.DataFrame(df_problem_and_human_solutions_list)

In [55]:
# Download the second dataset from Hugging Face Hub
file_path_2 = hf_hub_download(
    repo_id="facebook/BigOBench",
    filename="data/complexity_labels_light.jsonl",
    repo_type="dataset"
)

# Read the JSONL file and convert it to a DataFrame
df_complexity_labels_light = []
with open(file_path_2, "r", encoding="utf-8") as f:
    for line in f:
        try:
            df_complexity_labels_light.append(json.loads(line))
        except json.JSONDecodeError as e:
            print("Erorr JSON:", e)

df_complexity_labels_light = pd.DataFrame(df_complexity_labels_light)

In [ ]:
df_problem_and_human_solutions_list.head(5)

,problem_id,problem_name,description,correct_solution_list,data_source,source_specific_limits,codeforces_specific_metadata,tests,human_accuracy_rate,dataclass,complexity_framework
0,0,339_C. Xenia and Weights,{'text': 'Xenia has a set of weights and pan s...,"[{'solution_id': '0_0', 'solution_code': '__au...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 339, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '0000000101 3 ', '...",0.281633,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.40579710144927...
1,1,1547_E. Air Conditioners,{'text': 'On a strip of land of length n there...,"[{'solution_id': '1_0', 'solution_code': 'def ...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 1547, 'cf_index': 'E', 'cf_p...",{'public_tests': [{'input': '5 6 2 2 5 14 16 ...,0.620833,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.11409395973154...
2,2,268_C. Beautiful Sets of Points,{'text': 'Manao has invented a new mathematica...,"[{'solution_id': '2_0', 'solution_code': 'if _...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 268, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '2 2 ', 'output': ...",0.525066,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.05527638190954...
3,3,478_C. Table Decorations,"{'text': 'You have r red, g green and b blue b...","[{'solution_id': '3_0', 'solution_code': 'a = ...",CODEFORCES,"{'time_limit': {'seconds': 1, 'nanos': 0}, 'me...","{'cf_contest_id': 478, 'cf_index': 'C', 'cf_po...","{'public_tests': [{'input': '1 1 1 ', 'output'...",0.562264,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.41610738255033...
4,4,5_C. Longest Regular Bracket Sequence,{'text': 'This is yet another problem dealing ...,"[{'solution_id': '4_0', 'solution_code': 'stri...",CODEFORCES,"{'time_limit': {'seconds': 2, 'nanos': 0}, 'me...","{'cf_contest_id': 5, 'cf_index': 'C', 'cf_poin...","{'public_tests': [{'input': ')((())))(()()) ',...",0.395939,{'dataclass_code': 'import sys import time imp...,{'time_complexity_fail_rate': 0.03846153846153...


In [ ]:
df_complexity_labels_light.head(5)

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,0,339_C. Xenia and Weights,0_0,O(1),O(n**2),0.000019,0.012433
1,0,339_C. Xenia and Weights,0_1,None,None,NaN,NaN
2,0,339_C. Xenia and Weights,0_2,O(1),O(1),0.000004,584.000000
3,0,339_C. Xenia and Weights,0_3,None,None,NaN,NaN
4,0,339_C. Xenia and Weights,0_4,O(1),O(1),0.000013,323.000000


The attribute 'correst_solution_list' of the first dataset contains a list of code solutions for each sample. These solutions are stored as dictionaries containing as keys 'solution_id' and 'solution_code'. The aim is to obtain two new columns (attributes) in the final dataset one containing the solution id, and the other containing one solution code per sample. 

The Datasets results too big to be managed with the resources at our disposal, so it was decided to reduce the size of the datasets reducing the number of solutions for each description. After many considerations, it was choosen 300000 as the maximum amount of total samples for the final dataset. The following code reduces to 100 the maximum number of solutions for each description.

In [ ]:
# reduce num of possible correct solutions in correct_solution_list
max = 100  # num of corresct solutions kept

for problem in df_problem_and_human_solutions_list.correct_solution_list:
    if len(problem) > max:
        problem[:] = problem[:max]

In [ ]:
# check num of solutions:
for problem in df_problem_and_human_solutions_list.correct_solution_list[:30]:
    print('Number of samples:', len(problem))

Number of samples: 69
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 64
Number of samples: 75
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 54
Number of samples: 100
Number of samples: 100
Number of samples: 85
Number of samples: 100
Number of samples: 75
Number of samples: 86
Number of samples: 100
Number of samples: 90
Number of samples: 78
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 100
Number of samples: 98
Number of samples: 100
Number of samples: 91


Now we create a new DataFrame with columns 'solution_id' and 'solution_code' from the attribute 'correct_solution_list', and 'problem_description' from the atribute 'description' of the dataset df_problem_and_human_solutions_list.

In [ ]:
# modify correct_solutions_list to have two different attributes (solution_id and solution_code) insted of a list of them
solutions_list = df_problem_and_human_solutions_list.correct_solution_list.copy()
descriptions = df_problem_and_human_solutions_list.description.copy()

rows = []
for desc, solutions in zip(descriptions, solutions_list):
    for solution in solutions:
        rows.append({
            'problem_description': desc['text'],
            'solution_id': solution['solution_id'],
            'solution_code': solution['solution_code']
        })
new_matrix = pd.DataFrame(rows)

new_matrix.head(70)

,problem_description,solution_id,solution_code
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio..."
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...
...,...,...,...
65,Xenia has a set of weights and pan scales. Eac...,0_65,"p, m = [i for i, x in enumerate(input(), 1) if..."
66,Xenia has a set of weights and pan scales. Eac...,0_66,"s=input()\nA=[]\nfor i in range(1,11):\n if..."
67,Xenia has a set of weights and pan scales. Eac...,0_67,"def go(i, d, last):\n if i>0 and d<=0 or d>..."
68,Xenia has a set of weights and pan scales. Eac...,0_68,from sys import setrecursionlimit\nsetrecursio...


In [ ]:
new_matrix.shape

(293586, 3)

In [ ]:
new_matrix.describe()

,problem_description,solution_id,solution_code
count,293586,293586,293586
unique,3031,293586,289745
top,Hilbert's Hotel is a very unusual hotel since ...,0_0,"from sys import stdin, stdout\ninput = stdin.b..."
freq,200,1,2


Now the Dataset new_matrix and the second dataset are merged to abtain one line for each solution and keep only the solutions present in the new_matrix

In [ ]:
new_matrix.head(5)

,problem_description,solution_id,solution_code
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio..."
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...


In [ ]:
df_complexity_labels_light.head(5)

,problem_id,problem_name,solution_id,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,0,339_C. Xenia and Weights,0_0,O(1),O(n**2),0.000019,0.012433
1,0,339_C. Xenia and Weights,0_1,None,None,NaN,NaN
2,0,339_C. Xenia and Weights,0_2,O(1),O(1),0.000004,584.000000
3,0,339_C. Xenia and Weights,0_3,None,None,NaN,NaN
4,0,339_C. Xenia and Weights,0_4,O(1),O(1),0.000013,323.000000


In [ ]:
df = new_matrix.merge(df_complexity_labels_light, on='solution_id', how='left')

In [ ]:
df.head(5)

,problem_description,solution_id,solution_code,problem_id,problem_name,time_complexity_inferred,space_complexity_inferred,time_curve_coefficient,space_curve_coefficient
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,0,339_C. Xenia and Weights,O(1),O(n**2),0.000019,0.012433
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...,0,339_C. Xenia and Weights,None,None,NaN,NaN
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,0,339_C. Xenia and Weights,O(1),O(1),0.000004,584.000000
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio...",0,339_C. Xenia and Weights,None,None,NaN,NaN
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...,0,339_C. Xenia and Weights,O(1),O(1),0.000013,323.000000


In [ ]:
df.shape

(293586, 9)

Remove from df the atributes that we aren't going to use during the modeling

In [ ]:
df = df.drop(columns = ['problem_id', 'time_curve_coefficient', 'space_curve_coefficient'])

In [ ]:
df.head(5)

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,339_C. Xenia and Weights,O(1),O(n**2)
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...,339_C. Xenia and Weights,None,None
2,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,339_C. Xenia and Weights,O(1),O(1)
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio...",339_C. Xenia and Weights,None,None
4,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...,339_C. Xenia and Weights,O(1),O(1)


From the preliminary analysis we know that only the second dataset has na values. The na values are now eliminated form df

In [ ]:
# how many values of df are nan
df[df.isna().any(axis=1)]

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
1,Xenia has a set of weights and pan scales. Eac...,0_1,s = input()\nm = int(input())\nk1 = k2 = 0\nz ...,339_C. Xenia and Weights,None,None
3,Xenia has a set of weights and pan scales. Eac...,0_3,"from __future__ import division, print_functio...",339_C. Xenia and Weights,None,None
5,Xenia has a set of weights and pan scales. Eac...,0_5,import sys\nsys.setrecursionlimit(100000)\n\n\...,339_C. Xenia and Weights,None,None
7,Xenia has a set of weights and pan scales. Eac...,0_7,import sys\nsys.setrecursionlimit(1500)\ndef d...,339_C. Xenia and Weights,None,None
9,Xenia has a set of weights and pan scales. Eac...,0_9,s=input()\nm=int(input())\na=[]\nfor j in rang...,339_C. Xenia and Weights,None,None
...,...,...,...,...,...,...
293369,You are given an integer L. Construct a direct...,3102_83,import sys\ninput = sys.stdin.readline\n\nL = ...,p03267 AtCoder Beginner Contest 108 - All Your...,None,None
293422,You are given an integer sequence of length N....,3103_36,"from itertools import accumulate\nn, *A = map(...",p03739 AtCoder Beginner Contest 059 - Sequence,None,None
293431,You are given an integer sequence of length N....,3103_45,"n, *A = map(int, open(0).read().split())\ndef ...",p03739 AtCoder Beginner Contest 059 - Sequence,None,None
293446,You are given an integer sequence of length N....,3103_60,"n = int(input())\na = list(map(int,input().spl...",p03739 AtCoder Beginner Contest 059 - Sequence,None,None


In [ ]:
df = df.dropna()

In [ ]:
# size of the temp_df after removal of na values
df.shape

(244877, 6)

Rename the index

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head(3)

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,339_C. Xenia and Weights,O(1),O(n**2)
1,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,339_C. Xenia and Weights,O(1),O(1)
2,Xenia has a set of weights and pan scales. Eac...,0_4,# Target - Expert on CF\n# Be Humblefool\n\nim...,339_C. Xenia and Weights,O(1),O(1)


### Clean the code solutions
In the code of the solutions are present many elements that would render the training of models harder, such as comments (both inline and multiple lines).

Apon inspection of the code solutions, one code solution resulted not correct sintatically, so it was decided to drop it

In [ ]:
df.solution_code[114411]

'A,B = map(int,input().split(" "))\n\ns1 = "####################################################################################################\\n\\\n.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#\\n"\ns2 = "....................................................................................................\\n\\\n#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.#.\\n"\nblack = \'#\' * 100 + \'\\n\'\nwhite = \'.\' * 100 + \'\\n\'\n\ncnt = (A-1) // 50\nmer = (A - 1) % 50\nif mer == 0:\n\tup = s1 * cnt + black\nelse:\n\tup = s1 * cnt + black + (\'.#\' * mer) + (\'#\' * (100 - 2*mer)) + \'\\n\' + black\n\ncnt = (B-1) // 50\nmer = (B - 1) % 50\n\nif mer == 0:\n\tdown = s2 * cnt + white\nelse:\n\tdown = s2 * cnt + white + (\'#.\' * mer) + (\'.\' * (100 - 2*mer)) + \'\\n\' + white\n\nprint(str(len((up+down)) // 100) + " 100")\nprint(up + down)\n\n'

In [ ]:
df = df.drop(df.index[114411], axis = 0)

In [ ]:
df[114409:114413]

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
114409,You are given two integers A and B.\n\nPrint a...,1447_0,\nimport sys\nsys.setrecursionlimit(1 << 25)\n...,p03404 AtCoder Regular Contest 093 - Grid Comp...,O(1),O(n)
114410,You are given two integers A and B.\n\nPrint a...,1447_1,"from math import ceil\n\nA, B = map(int, input...",p03404 AtCoder Regular Contest 093 - Grid Comp...,O(n+m),O(n+m)
114412,You are given two integers A and B.\n\nPrint a...,1447_3,"a, b = map(int, input().split())\na -= 1\nb -=...",p03404 AtCoder Regular Contest 093 - Grid Comp...,O(nlogn),O(n**2)
114413,You are given two integers A and B.\n\nPrint a...,1447_4,"ab = list(map(int, input().split()))\n\ng = [[...",p03404 AtCoder Regular Contest 093 - Grid Comp...,O(1),O(1)


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.shape

(244876, 6)

Visualize inline comments

In [ ]:
import re
import nltk
from nltk import FreqDist

inline_comment_regex = '([^\"|^\']\#[^\'|^\"|^\n]*\n{1})'
concatenated_code = ' '.join(df.solution_code)
inline_comments_in_code = re.findall(inline_comment_regex, concatenated_code)
counts = nltk.FreqDist(inline_comments_in_code)
counts

FreqDist({'\n#\n': 2906, ' #!/usr/bin/env python3\n': 1645, ' # -*- coding: utf-8 -*-\n': 1553, '\n# region fastio\n': 1209, ' #\n': 1012, ' #!/usr/bin/env python\n': 693, '\n# endregion\n': 683, '\n#     else:\n': 545, ' # cook your dish here\n': 540, '\n# ------------------------------\n': 436, ...})

In [ ]:
for solution in df.solution_code:
    comment_code = re.findall(inline_comment_regex, solution)

In [ ]:
# print an example of code solution that contains inline comments
df.solution_code[65669]

'n = int(input())\ncont = 0\nmatriz = []\nfor i in range(n):\n    linha = list(input())\n    matriz.append(linha)\n    for i in range(n):\n        if linha[i] == \'#\':\n            cont+=1\nif cont%5 != 0:\n    print("NO")\nelse:\n    for i in range(0,n-1):\n        for j in range(1,n):\n            if matriz[i][j] == \'#\' and i < n-2 and j < n-1: # topo da cruz\n                if matriz[i+1][j] != \'#\' or matriz[i+2][j] != \'#\' or matriz[i+1][j-1] != \'#\' or matriz[i+1][j+1]!= \'#\':\n                    continue\n    \n                else:\n                    matriz[i][j] = \'.\'\n                    matriz[i+1][j] = \'.\'\n                    matriz[i+2][j] = \'.\'\n                    matriz[i+1][j-1] = \'.\'\n                    matriz[i+1][j+1] = \'.\'\n                    cont -= 5\n    if cont == 0:\n        print("YES")\n    else:\n        print("NO")\n            \n \t   \t\t\t  \t\t \t \t \t \t\t\t\t\t\t\t'

Remove inline comments

In [ ]:
for solution in df.solution_code:
    solution = re.sub(inline_comment_regex, '\n', solution)

In [ ]:
# print the solution code now without inline comments
df.solution_code[65669]

'n = int(input())\ncont = 0\nmatriz = []\nfor i in range(n):\n    linha = list(input())\n    matriz.append(linha)\n    for i in range(n):\n        if linha[i] == \'#\':\n            cont+=1\nif cont%5 != 0:\n    print("NO")\nelse:\n    for i in range(0,n-1):\n        for j in range(1,n):\n            if matriz[i][j] == \'#\' and i < n-2 and j < n-1: # topo da cruz\n                if matriz[i+1][j] != \'#\' or matriz[i+2][j] != \'#\' or matriz[i+1][j-1] != \'#\' or matriz[i+1][j+1]!= \'#\':\n                    continue\n    \n                else:\n                    matriz[i][j] = \'.\'\n                    matriz[i+1][j] = \'.\'\n                    matriz[i+2][j] = \'.\'\n                    matriz[i+1][j-1] = \'.\'\n                    matriz[i+1][j+1] = \'.\'\n                    cont -= 5\n    if cont == 0:\n        print("YES")\n    else:\n        print("NO")\n            \n \t   \t\t\t  \t\t \t \t \t \t\t\t\t\t\t\t'

Consider now multi-line comments

In [ ]:
multi_comment_regex = '([\'\"])\1\1[\d\D]*?\1{3}'
concatenated_code = ' '.join(df.solution_code)
multi_comments_in_code = re.findall(multi_comment_regex, concatenated_code)
counts = nltk.FreqDist(multi_comments_in_code)
counts

FreqDist({})

In [ ]:
for solution in df.solution_code:
    comment_code = re.findall(multi_comment_regex, solution)
    if comment_code:
        print(df.loc[df['solution_code'] == solution].index[0])

There are no multiline comments

### Save final Dataset in storage

In [ ]:
saving_path = path_to_project + '/final_ds.csv' if IN_COLAB else './final_ds.csv'
df.to_csv(saving_path, sep=',', index=False, encoding='utf-8')

## Transformers for Text Generation, Training and Fine-Tuning
In this section we are going to train and evaluate Transformers models to generate text.

### Libraries

Connect to Goodle Drive (if needed), add testing variables to the environment, and upload the dataset from storage

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import numpy as np

In [ ]:
dataset_path = path_to_project + '/final_ds.csv' if IN_COLAB else './final_ds.csv'
df = pd.read_csv(dataset_path)

In [ ]:
df.head(3)

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
0,Xenia has a set of weights and pan scales. Eac...,0_0,__author__ = 'ratnesh.mishra'\n\nweights = map...,339_C. Xenia and Weights,O(1),O(n**2)
1,Xenia has a set of weights and pan scales. Eac...,0_2,import sys\nsys.setrecursionlimit (1000000)\n\...,339_C. Xenia and Weights,O(1),O(1)
2,Xenia has a set of weights and pan scales. Eac...,0_4,import sys\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,339_C. Xenia and Weights,O(1),O(1)


### Train T5-Base

The developers of the Text-To-Text Transfer Transformer (T5) write:

'With T5, we propose reframing all NLP tasks into a unified text-to-text-format where the input and output are always text strings, in contrast to BERT-style models that can only output either a class label or a span of the input. Our text-to-text framework allows us to use the same model, loss function, and hyperparameters on any NLP task.'

The model has 223M parameters.

Import libraries that are going to be used in the following subsections

In [ ]:
!pip install -q transformers
!pip install datasets

In [ ]:
from datasets import Dataset, DatasetDict

#### Dataset Pre-processing



For the Model T5-Base only 8000 samples are kept from the dataset due to the limitation of the resources availabe to us. We decided to keep the training (and fine-tuning) time of the models at approximately 2 hours for this section, and the amount of samples allowed us to maintain this contraint.

In [ ]:
## only in initial part: reduce ds to allow faster testing of the code
df = df.head(8000).copy()
df.shape

(8000, 6)

Subset the data into training dataset, validation dataset and test dataset. We decided to use 10% of the dataset for tesing, and the remaining for train and validation.

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train_val, test_size=0.2)

In [ ]:
print('# train instances: ', train.shape[0])
print('# test instances:  ', test.shape[0])
print('# val instances:   ', val.shape[0])

# train instances:  5760
# test instances:   800
# val instances:    1440


We check the device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### Model and Tokenizer

Retrieve the model and the tokenizer

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
model_name = 'google-t5/t5-base'

Upload the Tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

Print some useful information on the tokenizer

In [ ]:
print("vocabulary size: ", tokenizer.vocab_size)

vocabulary size:  32000


In [ ]:
list(tokenizer.get_vocab().items())[600:610]

[('▁big', 600),
 ('▁God', 601),
 ('▁dass', 602),
 ('im', 603),
 ('▁30', 604),
 ('▁event', 605),
 ('▁development', 606),
 ('▁form', 607),
 ('▁read', 608),
 ('▁hand', 609)]

In [ ]:
text = "You have an array in input, order the elements in it in O(n) time complexity. Add a wrong wordd"
encoded_input = tokenizer._tokenize(text)
print(encoded_input)

['▁You', '▁have', '▁an', '▁array', '▁in', '▁input', ',', '▁order', '▁the', '▁elements', '▁in', '▁it', '▁in', '▁O', '(', 'n', ')', '▁time', '▁complexity', '.', '▁Add', '▁', 'a', '▁wrong', '▁word', 'd']


In [ ]:
encoded_ids = tokenizer(text)['input_ids']
print(encoded_ids)

[148, 43, 46, 5590, 16, 3785, 6, 455, 8, 2479, 16, 34, 16, 411, 599, 29, 61, 97, 11641, 5, 2334, 3, 9, 1786, 1448, 26, 1]


The model is uploaded and connected to the device

In [ ]:
t5 = T5ForConditionalGeneration.from_pretrained(model_name, device_map=device)
t5.resize_token_embeddings(len(tokenizer))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(32100, 768)

In [ ]:
print(t5)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

Consider now the number of parameters of the model

In [ ]:
n_params = sum(param.numel() for param in t5.parameters())
n_params

222882048

In [ ]:
for name, param in t5.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {param.size()}")
    print(f"Is trainable: {param.requires_grad}")
    print()

Parameter name: shared.weight
Parameter shape: torch.Size([32100, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.q.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.k.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.v.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.o.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
Parameter shape: torch.Size([32, 12])
Is trainable: True

Parameter name: encoder.block.0.layer.0.layer_norm.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: encoder.block.0.layer.1.DenseReluDense.wi.weight
Parameter shape: torch.Size([3072, 768])
Is trainable: True

Parameter name: encoder.block.0.la

#### Training


Now that we have both Tokenizer and Model we can tokenize the dataset and train the model

We obtain a single string for each conversation of problem description and solution, where the user input and the system response are separated by eos tokens.

In [ ]:
def apply_eos_token(idx, df, eos_token):
    # build the user input including the problem description, the time complexity and the space complexity
    chat_string = 'User:' + df.loc[idx, 'problem_description'] + ' Time complexity: ' + df.loc[idx, 'time_complexity_inferred'] + '; Space complexity: ' + df.loc[idx, 'space_complexity_inferred']
    # now add the eos token and the response from the assistant, the code solution for the problem
    chat_string = chat_string + eos_token + 'Assistant: ' + df.loc[idx, 'solution_code'] + eos_token
    return chat_string

In [ ]:
train_str = [apply_eos_token(idx, train, tokenizer.eos_token) for idx in train.index]
test_str = [apply_eos_token(idx, test, tokenizer.eos_token) for idx in test.index]
val_str = [apply_eos_token(idx, val, tokenizer.eos_token) for idx in val.index]

In [ ]:
train_str[0]

'User:It is known that the area of a regular dodecagon inscribed in a circle of radius a is 3a^2.\n\nGiven an integer r, find the area of a regular dodecagon inscribed in a circle of radius r.\n\nConstraints\n\n* 1 \\leq r \\leq 100\n* r is an integer.\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nr\n\n\nOutput\n\nPrint an integer representing the area of the regular dodecagon.\n\nExamples\n\nInput\n\n4\n\n\nOutput\n\n48\n\n\nInput\n\n15\n\n\nOutput\n\n675\n\n\nInput\n\n80\n\n\nOutput\n\n19200 Time complexity: O(1); Space complexity: O(1)</s>Assistant: N = int(input())\n\nprint(3*(N**2))</s>'

The strings where the eos token was applied, are now put in a dictionary structure

In [ ]:
train_data = Dataset.from_dict({'chat': train_str})
test_data = Dataset.from_dict({'chat': test_str})
val_data = Dataset.from_dict({'chat': val_str})

In [ ]:
data = DatasetDict()
data['train'] = train_data
data['val'] = val_data
data['test'] = test_data

The data is tokenized

In [ ]:
def tokenize_function(examples):
    input_encodings = tokenizer(examples["chat"],
        truncation=True,
        padding="max_length",
        max_length=512)
    sample = {
        'input_ids': input_encodings.input_ids,
        'attention_mask': input_encodings.attention_mask,
        'labels': input_encodings.input_ids.copy()
    }
    return sample

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/5760 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
# get all sequences in same batch
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Now the training starts

In [ ]:
from transformers import TrainingArguments, Trainer
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    "t5_trainer",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    learning_rate=1e-4,       # t5 needs a higher lr than other models
    lr_scheduler_type="linear",
    bf16=True,
    report_to=None,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model= t5,
    args= training_args,
    train_dataset= tokenized_data['train'],
    eval_dataset= tokenized_data['val'],
    data_collator= data_collator
)

Measure the time taken by the model to finish training

In [ ]:
import time
begin = time.time()

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,0.153200
1000,0.004200


TrainOutput(global_step=1080, training_loss=0.07308270597347506, metrics={'train_runtime': 7709.3416, 'train_samples_per_second': 2.241, 'train_steps_per_second': 0.14, 'total_flos': 1.05227923488768e+16, 'train_loss': 0.07308270597347506, 'epoch': 3.0})

In [ ]:
end = time.time()
print("Training time: ", end - begin)

Training time:  7709.833685159683


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(t5.named_parameters())

print('The T5 model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The T5 model has 257 different named parameters.

==== Embedding Layer ====

shared.weight                                           (32100, 768)
encoder.block.0.layer.0.SelfAttention.q.weight            (768, 768)

==== First Transformer ====

encoder.block.0.layer.0.SelfAttention.k.weight            (768, 768)
encoder.block.0.layer.0.SelfAttention.v.weight            (768, 768)
encoder.block.0.layer.0.SelfAttention.o.weight            (768, 768)
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight     (32, 12)
encoder.block.0.layer.0.layer_norm.weight                     (768,)
encoder.block.0.layer.1.DenseReluDense.wi.weight         (3072, 768)
encoder.block.0.layer.1.DenseReluDense.wo.weight         (768, 3072)
encoder.block.0.layer.1.layer_norm.weight                     (768,)
encoder.block.1.layer.0.SelfAttention.q.weight            (768, 768)
encoder.block.1.layer.0.SelfAttention.k.weight            (768, 768)
encoder.block.1.layer.0.SelfAttention.v.weight      

In [ ]:
t5.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

Save model's parameters

In [ ]:
from datetime import datetime
t5_training_path = path_to_project + '/Transformer-trained-models/' + f"t5_train_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
tokenizer.save_pretrained(t5_training_path)
t5.save_pretrained(t5_training_path)
print(f"Checkpoint saved at: \'{t5_training_path}\'")

Checkpoint saved at: '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/t5_train_2025_05_18_12_35_46'


#### Testing

Retrieve the trained model

In [ ]:
device = 'cuda'

#t5_training_path = '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/t5_train_2025_05_18_12_35_46'
tokenizer = T5Tokenizer.from_pretrained(t5_training_path)
t5 = T5ForConditionalGeneration.from_pretrained(t5_training_path, device_map=device)

To test the model, we first extract one conversation from the test data randomly, give it as input to the model, and we see the response

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

39
User: Vlad likes to eat in cafes very much. During his life, he has visited cafes n times. Unfortunately, Vlad started to feel that his last visits are not any different from each other. To fix that Vlad had a small research.

First of all, Vlad assigned individual indices to all cafes. Then, he wrote down indices of cafes he visited in a row, in order of visiting them. Now, Vlad wants to find such a cafe that his last visit to that cafe was before his last visits to every other cafe. In other words, he wants to find such a cafe that he hasn't been there for as long as possible. Help Vlad to find that cafe.

Input

In first line there is one integer n (1 ≤ n ≤ 2·105) — number of cafes indices written by Vlad.

In second line, n numbers a1, a2, ..., an (0 ≤ ai ≤ 2·105) are written — indices of cafes in order of being visited by Vlad. Vlad could visit some cafes more than once. Note that in numeration, some indices could be omitted.

Output

Print one integer — index of the cafe that 

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, en = dialogue.split('</s>')

# add eos token at the end of test_input and test_output
test_input = test_input + tokenizer.eos_token + ' Assistant: '
test_output = test_output + tokenizer.eos_token

print('User input: \n', test_input)
print('User input lenght: ', len(test_input))
print('####')
print('Correct solution output: \n', test_output)

User input: 
 User: Vlad likes to eat in cafes very much. During his life, he has visited cafes n times. Unfortunately, Vlad started to feel that his last visits are not any different from each other. To fix that Vlad had a small research.

First of all, Vlad assigned individual indices to all cafes. Then, he wrote down indices of cafes he visited in a row, in order of visiting them. Now, Vlad wants to find such a cafe that his last visit to that cafe was before his last visits to every other cafe. In other words, he wants to find such a cafe that he hasn't been there for as long as possible. Help Vlad to find that cafe.

Input

In first line there is one integer n (1 ≤ n ≤ 2·105) — number of cafes indices written by Vlad.

In second line, n numbers a1, a2, ..., an (0 ≤ ai ≤ 2·105) are written — indices of cafes in order of being visited by Vlad. Vlad could visit some cafes more than once. Note that in numeration, some indices could be omitted.

Output

Print one integer — index of the

We generate the response from the model

In [ ]:
input_ids = tokenizer(test_input, return_tensors="pt", max_length = 2000).to(device)
output = t5.generate(**input_ids, max_new_tokens=800)
gen_text = tokenizer.decode(output[0])

print(gen_text)
print(gen_text.split('Assistant:')[-1])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<pad> User: Vlad likes to eat in cafes very much. During his life, he has visited cafes n times. Unfortunately, Vlad started to feel that his last visits are not any different from each other. To fix that Vlad had a small research. First of all, Vlad assigned individual indices to all cafes. Then, he wrote down indices of cafes he visited in a row, in order of visiting them. Now, Vlad wants to find such a cafe that his last visit to that cafe was before his last visits to every other cafe. In other words, he wants to find such a cafe that he hasn't been there for as long as possible. Help Vlad to find that cafe. Input In first line there is one integer n (1 <unk> n <unk> 2<unk> 105) — number of cafes indices written by Vlad. In second line, n numbers a1, a2, ..., an (0 <unk> ai <unk> 2<unk> 105) are written — indices of cafes in order of being visited by Vlad. Vlad could visit some cafes more than once. Note that in numeration, some indices could be omitted. Output Print one integer — 

The model does not perform very well and struggles to produce code in output and to maintain the conversation format given to it. What it tries to do is to describe the solution process without explicitly generating code. 

We expected this type of reults, the T5 model has very good performance on translation and sumarization, while it wasn't trained as a chatbot to produce code initially. Moreover, due to our limited resources, 8000 samples are too little to train such a big model (223M parameters) in an efective way.

#### Evaluation Metrics

It was decided to consider the metrics Perplexity, BLEU and F1.

**Perplexity**

We select only the first three samples of the test set due to limitations in the RAM available to us

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('</s>')[0] + tokenizer.eos_token + 'Assistant: ' for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('</s>')[1] + tokenizer.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
!pip install lmppl

In [ ]:
import lmppl

scorer = lmppl.EncoderDecoderLM(t5_training_path)

ppl = scorer.get_perplexity(input_texts= test_input, output_texts=test_output)
print(list( ppl))
print(f"average perplexity: {sum(ppl)/len(ppl)}")

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/configuration_auto.py:1099: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4056: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Passing a tuple of `past_key_va

[23387.28944204467, 1260.3922689555861, 22569.483405842497]
average perplexity: 15739.055038947583


In [ ]:
print(f"user input: \n{test_input[ppl.index(min(ppl))]}")
print('#################')
print(f"prediction: \n{test_output[ppl.index(min(ppl))]}")

user input: 
User: Gleb ordered pizza home. When the courier delivered the pizza, he was very upset, because several pieces of sausage lay on the crust, and he does not really like the crust.

The pizza is a circle of radius r and center at the origin. Pizza consists of the main part — circle of radius r - d with center at the origin, and crust around the main part of the width d. Pieces of sausage are also circles. The radius of the i -th piece of the sausage is ri, and the center is given as a pair (xi, yi).

Gleb asks you to help determine the number of pieces of sausage caught on the crust. A piece of sausage got on the crust, if it completely lies on the crust.

Input

First string contains two integer numbers r and d (0 ≤ d < r ≤ 500) — the radius of pizza and the width of crust.

Next line contains one integer number n — the number of pieces of sausage (1 ≤ n ≤ 105).

Each of next n lines contains three integer numbers xi, yi and ri ( - 500 ≤ xi, yi ≤ 500, 0 ≤ ri ≤ 500), where x

**BLEU**

BLEU focuses on precision by counting matching n-grams.

In [ ]:
!pip -q install parlai

In [ ]:
from parlai.core.metrics import BleuMetric

input_ids = tokenizer(test_input[0], return_tensors="pt", max_length = 2000).to(device)
output = t5.generate(**input_ids, max_new_tokens=800)
gen_text = tokenizer.decode(output[0])

bleu = BleuMetric.compute(gen_text, [test_output[0]])
print(f"BLEU: {bleu}")

BLEU: 3.804e-09


The Bleu metric has a very low value, indicating very little overlap between the generated text and the solution of the test sample 

**F1 Score**

In [ ]:
from parlai.core.metrics import F1Metric

f1_score = F1Metric.compute(gen_text, [test_output[0]])
print(f"F1: {f1_score}")

F1: 0.03226


### Fine-tuning T5-small


**Discaimer:** We decided to not include the whole code for this section: the code was very similar to the one used to train the T5-base model and, given the already very long notebook and the not-excellent performance after the fine-tuning, we considered best to not include it.

The T5-small model has 60M parameters, and is part of the family of the T5 text-to-text models. It was trained on Common Crawl.
For the fine-tuning of T5-small we decided to keep 5000 samples. The model is relatively small, with only 60M parameters and already trained: a higher amount of samples led to worse performance because it caused the training to overwrite parameters unecessarily.

#### Model and Tokenizer
We include the model specifics:

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model

In [ ]:
model_name = 'google-t5/t5-small'

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
t5 = T5ForConditionalGeneration.from_pretrained(model_name, device_map=device)
t5.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Embedding(32100, 512)

In [ ]:
print(t5)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

#### Results

The perplexity calculated before the fine-tuning was: [66.54613901709006, 105.79112153990572, 35432.329456935135] with
average perplexity: 11868.222239164044.

The training lasted 1314.5914108753204 seconds.

We then tested the text generation of the fine-tuned model on a random sample from the test set: also in this case, the model does not perform well. It doesn't generate the code, mainly copies the input, and struggles to keep the separation between 'User' and 'Assistant'. Overall the generated text from T5-small is less clear than the text generated by the training of T5-base: the bigger amount of parameters of T5-base guarantees better text quality.

We considered the evaluation metrics Perplexity, Bleu and F1.

**Perplexity:** it was calculated considering three samples form the test set, the results are: [439.4691293912123, 720.7729464902942, 1981.2242516858746] with average perplexity: 1047.1554425224604. We can see that the fine-tuning did not increase considerably the performance of the model. The average of the perplexity decreased though, indicating a better understanding of the new functionality with respect to the model prior the fine-tuning.

**Bleu:** 7.417e-06

**F1-score:** 0.0744

Also Bleu and F1 scores are extremely low.

### Fine-tuning gpt2-small

GPT-2 is a transformers model pretrained on a very large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts. More precisely, it was trained to guess the next word in sentences.

Moreover, inputs are sequences of continuous text of a certain length and the targets are the same sequence, shifted one token (word or piece of word) to the right. The model uses internally a mask-mechanism to make sure the predictions for the token i only uses the inputs from 1 to i but not the future tokens.

This is the smallest version of GPT-2, with 124M parameters. 

Connect to Goodle Drive and upload the dataset

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import numpy as np

In [ ]:
dataset_path = path_to_project + '/final_ds.csv' if IN_COLAB else '/final_ds.csv'
df = pd.read_csv(dataset_path)

Import libraries that are going to be used in the following subsections

In [ ]:
!pip install -q transformers
!pip install datasets

In [ ]:
from datasets import Dataset, DatasetDict

#### Dataset Pre-processing



For the fine-tuning of gpt2-small we decided to keep 15000 samples. The training lasts approximately 2 hours, and, given the size and the pre-training of the model, gpt2 had better performance with a bigger amount of samples.

In [ ]:
df = df.head(15000).copy()
df.shape

Subset the data into training dataset, validation dataset and test dataset. We decided to use 10% of the dataset for tesing, and the remaining for train and validation.

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train_val, test_size=0.2)

In [ ]:
print('# train instances: ', train.shape[0])
print('# test instances:  ', test.shape[0])
print('# val instances:   ', val.shape[0])

# train instances:  10800
# test instances:   1500
# val instances:    2700


Now we check the device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### Model and Tokenizer

Retrieve the model and the tokenizer

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
model_name = 'gpt2'

Upload the Tokenizer

In [ ]:
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

In [ ]:
print("vocabulary size: ", tokenizer_gpt2.vocab_size)

vocabulary size:  50257


In [ ]:
list(tokenizer_gpt2.get_vocab().items())[600:610]

[('int', 600),
 ('ress', 601),
 ('ations', 602),
 ('ail', 603),
 ('Ġ4', 604),
 ('ical', 605),
 ('Ġthem', 606),
 ('Ġher', 607),
 ('ount', 608),
 ('ĠCh', 609)]

In [ ]:
text = "You have an array in input, order the elements in it in O(n) time complexity. Add a wrong wordd"
encoded_input = tokenizer_gpt2._tokenize(text)
print(encoded_input)

['You', 'Ġhave', 'Ġan', 'Ġarray', 'Ġin', 'Ġinput', ',', 'Ġorder', 'Ġthe', 'Ġelements', 'Ġin', 'Ġit', 'Ġin', 'ĠO', '(', 'n', ')', 'Ġtime', 'Ġcomplexity', '.', 'ĠAdd', 'Ġa', 'Ġwrong', 'Ġword', 'd']


In [ ]:
encoded_ids = tokenizer_gpt2(text)['input_ids']
print(encoded_ids)

[1639, 423, 281, 7177, 287, 5128, 11, 1502, 262, 4847, 287, 340, 287, 440, 7, 77, 8, 640, 13357, 13, 3060, 257, 2642, 1573, 67]


It was necessary to use GPT2LMHeadModel instead of GPT2MOdel, beacuse the latter is not compatible with the Trainer due to a mismatch of the arguments.

In [ ]:
gpt2 = GPT2LMHeadModel.from_pretrained(model_name, device_map=device)
gpt2.resize_token_embeddings(len(tokenizer_gpt2))

Embedding(50257, 768)

In [ ]:
print(gpt2)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


Consider now the number of parameters of the model

In [ ]:
n_params = sum(param.numel() for param in gpt2.parameters())
n_params

124439808

In [ ]:
for name, param in gpt2.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {param.size()}")
    print(f"Is trainable: {param.requires_grad}")
    print()

Parameter name: transformer.wte.weight
Parameter shape: torch.Size([50257, 768])
Is trainable: True

Parameter name: transformer.wpe.weight
Parameter shape: torch.Size([1024, 768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.weight
Parameter shape: torch.Size([768, 2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.bias
Parameter shape: torch.Size([2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.bias
Parameter shape: torch.Size([7

#### Training


Now that we have both Tokenizer and Model we can tokenize the dataset and train the model

The eos tokens have to be added by hand to obtain a single string for each 'conversation'. In a single string we include the user input, that includes the description of the problem, and the time and space complexities, and the espected answer from the assistant.

In [ ]:
def apply_eos_token(idx, df, eos_token):
    # build the user input including the problem description, the time complexity and the space complexity
    chat_string = 'User:' + df.loc[idx, 'problem_description'] + ' Time complexity: ' + df.loc[idx, 'time_complexity_inferred'] + '; Space complexity: ' + df.loc[idx, 'space_complexity_inferred']
    # now add the eos token and the response from the assistant, the code solution for the problem
    chat_string = chat_string + eos_token + 'Assistant: ' + df.loc[idx, 'solution_code'] + eos_token
    return chat_string

In [ ]:
train_str = [apply_eos_token(idx, train, tokenizer_gpt2.eos_token) for idx in train.index]
test_str = [apply_eos_token(idx, test, tokenizer_gpt2.eos_token) for idx in test.index]
val_str = [apply_eos_token(idx, val, tokenizer_gpt2.eos_token) for idx in val.index]

In [ ]:
train_str[0]

"User: Naruto has sneaked into the Orochimaru's lair and is now looking for Sasuke. There are T rooms there. Every room has a door into it, each door can be described by the number n of seals on it and their integer energies a_1, a_2, ..., a_n. All energies a_i are nonzero and do not exceed 100 by absolute value. Also, n is even.\n\nIn order to open a door, Naruto must find such n seals with integer energies b_1, b_2, ..., b_n that the following equality holds: a_{1} ⋅ b_{1} + a_{2} ⋅ b_{2} + ... + a_{n} ⋅ b_{n} = 0. All b_i must be nonzero as well as a_i are, and also must not exceed 100 by absolute value. Please find required seals for every room there.\n\nInput\n\nThe first line contains the only integer T (1 ≤ T ≤ 1000) standing for the number of rooms in the Orochimaru's lair. The other lines contain descriptions of the doors.\n\nEach description starts with the line containing the only even integer n (2 ≤ n ≤ 100) denoting the number of seals.\n\nThe following line contains the s

The strings where the eos token was applied, are now put in a dictionary structure

In [ ]:
train_data = Dataset.from_dict({'chat': train_str})
test_data = Dataset.from_dict({'chat': test_str})
val_data = Dataset.from_dict({'chat': val_str})

In [ ]:
data = DatasetDict()
data['train'] = train_data
data['val'] = val_data
data['test'] = test_data

Tokenize the data

In [ ]:
def tokenize_function(examples):
    input_encodings = tokenizer_gpt2(examples["chat"],
        truncation=True,
        padding="max_length",
        max_length=512)
    sample = {
        'input_ids': input_encodings.input_ids,
        'attention_mask': input_encodings.attention_mask,
        'labels': input_encodings.input_ids.copy()
    }
    return sample

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/10800 [00:00<?, ? examples/s]

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# get all sequences in same batch
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer_gpt2, mlm=False)

Now the training starts

#### Calculate Perplexity before fine-tuning.

Get a random conversation from the test set

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

78
User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the first example,

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

test_input = test_input + tokenizer_gpt2.eos_token + ' Assistant: '
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: ', test_input)
print('####')
print('Correct solution output: ', test_output)

User input:  User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the firs

We decided to use pipeline from the transformers library to generate the output of the model, indicating the task 'text-generation'

In [ ]:
!pip install pipeline transformers evaluate
from transformers import pipeline
import evaluate

In [ ]:
pipe = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

Device set to use cuda


In [ ]:
gen_output = pipe(test_input, max_new_tokens=256)
print("Generated text:\n", gen_output[0]["generated_text"])

Generated text:
 User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the 

The generated text has nothing to do with python code and includes references to pop culture, probably included in the pre-training of the model

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity_before_finetuning = perplexity_metric.compute(
    predictions= test_input,
    add_start_token=False,
    model_id= 'gpt2'
)
print("Perplexity before fine-tuning:", perplexity_before_finetuning)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity before fine-tuning: {'perplexities': [28.182695388793945, 14.845183372497559, 14.715679168701172], 'mean_perplexity': np.float64(19.24785264333089)}


#### Fine-Tuning
Now we can start the fine-tuning

In [ ]:
from transformers import TrainingArguments, Trainer
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    "gpt2_trainer",
    #evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8, 
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    learning_rate=6.25e-5,
    lr_scheduler_type="linear",
    bf16=True,  
    report_to=None,  
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model= gpt2,
    args= training_args,
    train_dataset= tokenized_data['train'],
    eval_dataset= tokenized_data['val'],
    data_collator= data_collator
)

Measure the time taken by the model to finish training

In [ ]:
import time
begin = time.time()

In [ ]:
trainer.train()

Step,Training Loss
500,1.133900
1000,0.341100
1500,0.236900
2000,0.211400


TrainOutput(global_step=2025, training_loss=0.4775619372615108, metrics={'train_runtime': 7315.882, 'train_samples_per_second': 4.429, 'train_steps_per_second': 0.277, 'total_flos': 8465861836800000.0, 'train_loss': 0.4775619372615108, 'epoch': 3.0})

In [ ]:
end = time.time()
print("Training time: ", end - begin)

Training time:  7316.4667019844055


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(gpt2.named_parameters())

print('The GPT-2 model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The GPT-2 model has 148 different named parameters.

==== Embedding Layer ====

transformer.wte.weight                                  (50257, 768)
transformer.wpe.weight                                   (1024, 768)

==== First Transformer ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072

In [ ]:
gpt2.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Save model's parameters

In [ ]:
from datetime import datetime

gpt2_finetune_path = path_to_project + '/Transformer-trained-models/' + f"gpt2_fine_tuning_(chat+code)_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
tokenizer_gpt2.save_pretrained(gpt2_finetune_path)
gpt2.save_pretrained(gpt2_finetune_path)
print(f"Checkpoint saved at: \'{gpt2_finetune_path}\'")

Checkpoint saved at: '/content/drive/MyDrive/UnB/NLP_Project/Transformer-trained-models/gpt2_fine_tuning_(chat+code)_2025_05_15_13_50_58'


#### Testing

Retrieve the trained model

In [ ]:
!pip install evaluate
import evaluate

In [ ]:
device = 'cuda'

gpt2_finetune_path = '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/gpt2_fine_tuning_(chat+code)_2025_05_15_13_50_58'
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(gpt2_finetune_path)
gpt2 = GPT2LMHeadModel.from_pretrained(gpt2_finetune_path, device_map=device)

To test the model, we first extract one chat from the test data randomly, give it as input to the model, and we see the response

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

78
User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the first example,

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

# add eos token at the end of test_input and test_output
test_input = test_input + tokenizer_gpt2.eos_token + ' Assistant: '
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: \n', test_input)
print('####')
print('Correct solution output: \n', test_output)

User input: 
 User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Output


3

Note

In the fir

In [ ]:
!pip install pipeline transformers
from transformers import pipeline

pipe_finetuned = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

In [ ]:
gen_output = pipe_finetuned(test_input, max_new_tokens=256)
print("Generated text after the fine-tuning: \n", gen_output[0]["generated_text"])

Generated text after the fine-tuning: 
 User: Pavel has several sticks with lengths equal to powers of two.

He has a_0 sticks of length 2^0 = 1, a_1 sticks of length 2^1 = 2, ..., a_{n-1} sticks of length 2^{n-1}. 

Pavel wants to make the maximum possible number of triangles using these sticks. The triangles should have strictly positive area, each stick can be used in at most one triangle.

It is forbidden to break sticks, and each triangle should consist of exactly three sticks.

Find the maximum possible number of triangles.

Input

The first line contains a single integer n (1 ≤ n ≤ 300 000) — the number of different lengths of sticks.

The second line contains n integers a_0, a_1, ..., a_{n-1} (1 ≤ a_i ≤ 10^9), where a_i is the number of sticks with the length equal to 2^i.

Output

Print a single integer — the maximum possible number of non-degenerate triangles that Pavel can make.

Examples

Input


5
1 2 2 2 2


Output


3


Input


3
1 1 1


Output


0


Input


3
3 3 3


Ou

As it can be noticed, after the fine-tuning the model is able to generate python code. The generated text starts with a reference to the user input, but it able to go forward and include an answer from the 'Assistant'. The model is not yet able to generate a correct solution for the problem, but given the smaller size of gpt2-small, it was expected.

#### Evaluation Metrics

It was decided to consider the metrics Perplexity and BLEU.

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity = perplexity_metric.compute(
    predictions= test_input,
    add_start_token=False,
    model_id= gpt2_finetune_path
)
print("Perplexity post fine-tuning:", perplexity)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity post fine-tuning: {'perplexities': [1.0604777336120605, 1.047598958015442, 1.0766164064407349], 'mean_perplexity': np.float64(1.0615643660227458)}


The perplexity calculated before the fine-tuning was [28.182695388793945, 14.845183372497559, 14.715679168701172] with mean_perplexity: np.float64(19.24785264333089)

We can see, now, that the perplexity values have sistematically decreased, indicating that the fine-tuning  process was able to improve the performance of the model on this specific task.

**BLEU**

BLEU focuses on precision by counting matching n-grams.

In [ ]:
# BLEU Score
bleu_metric = evaluate.load("bleu")
bleu_results = bleu_metric.compute(
    predictions= [pipe_finetuned(test_input[0], max_new_tokens=256)[0]['generated_text']], 
    references= [test_output[0]]  
)
print("BLEU Score:", bleu_results)

BLEU Score: {'bleu': 0.12839728594651886, 'precisions': [0.3142857142857143, 0.14537444933920704, 0.09271523178807947, 0.06415929203539823], 'brevity_penalty': 1.0, 'length_ratio': 1.6666666666666667, 'translation_length': 455, 'reference_length': 273}


The Bleu scores are still low overall, but are much higher than the Bleu scores for the T5 models, further indicating how the better performance among similar sized model was achieved by the gpt2 model

### Fine-tuning gpt2-CodeParrot

This model is a base GPT-2 architecture with 124M parameters. It was trained on the huggingface-course/codeparrot-ds-valid dataset, which is a small subset of the original WebText dataset used to train GPT-2, and includes python code.

Connect to Goodle Drive and upload the dataset

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/NLP_Project'
  sys.path.append(path_to_project)
  IN_COLAB = True
except:
  IN_COLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
import numpy as np

In [ ]:
dataset_path = path_to_project + '/final_ds.csv' if IN_COLAB else '/final_ds.csv'
df = pd.read_csv(dataset_path)

Import libraries that are going to be used in the following subsections

In [ ]:
!pip install -q transformers
!pip install datasets

In [ ]:
from datasets import Dataset, DatasetDict

#### Dataset Pre-processing



Considering the small size of the model and the limited data used during its training, we decided to reduce the number of samples used for the fine-tuning to avoid overwriting parameters unecessarily and degrade the performance of the model

In [ ]:
df = df.head(5000).copy()
df.shape

Subset the data into training dataset, validation dataset and test dataset. We decided to use 10% of the dataset for tesing, and the remaining for train and validation.

In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train_val, test_size=0.2)

In [ ]:
print('# train instances: ', train.shape[0])
print('# test instances:  ', test.shape[0])
print('# val instances:   ', val.shape[0])

# train instances:  3600
# test instances:   500
# val instances:    900


Now we check the device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### Model and Tokenizer

Retreiev the model and the tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = 'kailasps/GPT2-codeparrot'

Upload the Tokenizer

In [ ]:
tokenizer_gpt2 = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [ ]:
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

In [ ]:
print("vocabulary size: ", tokenizer_gpt2.vocab_size)

vocabulary size:  50000


In [ ]:
list(tokenizer_gpt2.get_vocab().items())[600:610]

[('ĠSources', 46176),
 ('Ġcac', 33927),
 ('Ġsegm', 31074),
 ('electric', 30949),
 ('Ġuntagged', 43874),
 ('ellipse', 22936),
 ('ĠSubscription', 26798),
 ('Ġlooping', 24598),
 ('`', 64),
 ('HierarchySession', 26069)]

In [ ]:
text = "You have an array in input, order the elements in it in O(n) time complexity. Add a wrong wordd"
encoded_input = tokenizer_gpt2.tokenize(text)
print(encoded_input)

['You', 'Ġhave', 'Ġan', 'Ġarray', 'Ġin', 'Ġinput', ',', 'Ġorder', 'Ġthe', 'Ġelements', 'Ġin', 'Ġit', 'Ġin', 'ĠO', '(', 'n', ')', 'Ġtime', 'Ġcomplexity', '.', 'ĠAdd', 'Ġa', 'Ġwrong', 'Ġword', 'd']


In [ ]:
encoded_ids = tokenizer_gpt2(text)['input_ids']
print(encoded_ids)

[6147, 1054, 309, 960, 253, 935, 12, 1444, 256, 2347, 253, 577, 253, 690, 8, 78, 9, 626, 16989, 14, 1822, 231, 6401, 1975, 68]


The model is uploaded and connected to the device


In [ ]:
gpt2 = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)
gpt2.resize_token_embeddings(len(tokenizer_gpt2))

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Embedding(50000, 768)

In [ ]:
print(gpt2)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)


Consider now the number of parameters of the model

In [ ]:
n_params = sum(param.numel() for param in gpt2.parameters())
n_params

124242432

In [ ]:
for name, param in gpt2.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter shape: {param.size()}")
    print(f"Is trainable: {param.requires_grad}")
    print()

Parameter name: transformer.wte.weight
Parameter shape: torch.Size([50000, 768])
Is trainable: True

Parameter name: transformer.wpe.weight
Parameter shape: torch.Size([1024, 768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_1.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.weight
Parameter shape: torch.Size([768, 2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_attn.bias
Parameter shape: torch.Size([2304])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.weight
Parameter shape: torch.Size([768, 768])
Is trainable: True

Parameter name: transformer.h.0.attn.c_proj.bias
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.weight
Parameter shape: torch.Size([768])
Is trainable: True

Parameter name: transformer.h.0.ln_2.bias
Parameter shape: torch.Size([7

#### Training


Now that we have both Tokenizer and Model we can tokenize the dataset and train the model

In [ ]:
def apply_eos_token(idx, df, eos_token):
    # build the user input including the problem description, the time complexity and the space complexity
    chat_string = 'User:' + df.loc[idx, 'problem_description'] + ' Time complexity: ' + df.loc[idx, 'time_complexity_inferred'] + '; Space complexity: ' + df.loc[idx, 'space_complexity_inferred']
    # now add the eos token and the response from the assistant, the code solution for the problem
    chat_string = chat_string + eos_token + 'Assistant: ' + df.loc[idx, 'solution_code'] + eos_token
    return chat_string

In [ ]:
train_str = [apply_eos_token(idx, train, tokenizer_gpt2.eos_token) for idx in train.index]
test_str = [apply_eos_token(idx, test, tokenizer_gpt2.eos_token) for idx in test.index]
val_str = [apply_eos_token(idx, val, tokenizer_gpt2.eos_token) for idx in val.index]

In [ ]:
train_str[0]

'User: Manao has invented a new mathematical term — a beautiful set of points. He calls a set of points on a plane beautiful if it meets the following conditions:\n\n  1. The coordinates of each point in the set are integers. \n  2. For any two points from the set, the distance between them is a non-integer. \n\n\n\nConsider all points (x, y) which satisfy the inequations: 0 ≤ x ≤ n; 0 ≤ y ≤ m; x + y > 0. Choose their subset of maximum size such that it is also a beautiful set of points.\n\nInput\n\nThe single line contains two space-separated integers n and m (1 ≤ n, m ≤ 100).\n\nOutput\n\nIn the first line print a single integer — the size k of the found beautiful set. In each of the next k lines print a pair of space-separated integers — the x- and y- coordinates, respectively, of a point from the set.\n\nIf there are several optimal solutions, you may print any of them.\n\nExamples\n\nInput\n\n2 2\n\n\nOutput\n\n3\n0 1\n1 2\n2 0\n\n\nInput\n\n4 3\n\n\nOutput\n\n4\n0 3\n2 1\n3 0\n4 

The strings where the eos token was applied, are now put in a dictionary structure

In [ ]:
train_data = Dataset.from_dict({'chat': train_str})
test_data = Dataset.from_dict({'chat': test_str})
val_data = Dataset.from_dict({'chat': val_str})

In [ ]:
data = DatasetDict()
data['train'] = train_data
data['val'] = val_data
data['test'] = test_data

Tokenize the data

In [ ]:
def tokenize_function(examples):
    input_encodings = tokenizer_gpt2(examples["chat"],
        truncation=True,
        padding="max_length",
        max_length=512)
    sample = {
        'input_ids': input_encodings.input_ids,
        'attention_mask': input_encodings.attention_mask,
        'labels': input_encodings.input_ids.copy()
    }
    return sample

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# get all sequences in same batch
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer_gpt2, mlm=False)

Now the training starts

##### Calculate Perplexity before fine-tuning.

Get a random conversation from the test set

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

19
User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer t (1 ≤ t ≤

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

test_input = test_input + tokenizer_gpt2.eos_token + ' Assistant: '
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: ', test_input)
print('####')
print('Correct solution output: ', test_output)

User input:  User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer 

In [ ]:
!pip install pipeline transformers evaluate
from transformers import pipeline
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
pipe = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

Device set to use cuda


In [ ]:
gen_output = pipe(test_input, max_new_tokens=256)
print("Generated text:\n", gen_output[0]["generated_text"])

Generated text:
 User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single inte

The generated text does not include the solution code

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity_before_finetuning = perplexity_metric.compute(
    predictions= test_input,
    model_id= model_name
)
print("Perplexity before fine-tuning:", perplexity_before_finetuning)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity before fine-tuning: {'perplexities': [653.0138549804688, 570.7035522460938, 424.50201416015625], 'mean_perplexity': np.float64(549.4064737955729)}


#### Fine-Tuning
Now we can start the fine-tuning

In [ ]:
from transformers import TrainingArguments, Trainer
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    "gpt2-parrot_trainer",
    #evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8, 
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    learning_rate=6.25e-5,
    lr_scheduler_type="linear",
    bf16=True,  
    report_to=None, 
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model= gpt2,
    args= training_args,
    train_dataset= tokenized_data['train'],
    eval_dataset= tokenized_data['val'],
    data_collator= data_collator
)

Measure the time taken by the model to finish training

In [ ]:
import time
begin = time.time()

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.458700


TrainOutput(global_step=675, training_loss=1.1649107078269676, metrics={'train_runtime': 2385.8294, 'train_samples_per_second': 4.527, 'train_steps_per_second': 0.283, 'total_flos': 2821953945600000.0, 'train_loss': 1.1649107078269676, 'epoch': 3.0})

In [ ]:
end = time.time()
print("Training time: ", end - begin)

Training time:  2386.2620153427124


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(gpt2.named_parameters())

print('The GPT-2 model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The GPT-2 model has 148 different named parameters.

==== Embedding Layer ====

transformer.wte.weight                                  (50000, 768)
transformer.wpe.weight                                   (1024, 768)

==== First Transformer ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072

In [ ]:
gpt2.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

Save model's parameters

In [ ]:
from datetime import datetime

gpt2_finetune_path = path_to_project + '/Transformer-trained-models/' + f"gpt2-parrot_fine_tuning_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
tokenizer_gpt2.save_pretrained(gpt2_finetune_path)
gpt2.save_pretrained(gpt2_finetune_path)
print(f"Checkpoint saved at: \'{gpt2_finetune_path}\'")

Checkpoint saved at: '/content/drive/MyDrive/SFX/NLP_Project/Transformer-trained-models/gpt2-parrot_fine_tuning_2025_05_16_14_58_31'


#### Testing

Retrieve the trained model

In [ ]:
!pip install evaluate
import evaluate

In [ ]:
device = 'cuda'

#gpt2_finetune_path = '/content/drive/MyDrive/NLP_Project/Transformer-trained-models/gpt2-parrot_fine_tuning_2025_05_16_14_58_31'
tokenizer_gpt2 = AutoTokenizer.from_pretrained(gpt2_finetune_path)
gpt2 = AutoModelForCausalLM.from_pretrained(gpt2_finetune_path, device_map = device)

To test the model, we first extract one chat from the test data randomly, give it as input to the model, and we see the response

In [ ]:
import random

random.seed(43)

idx = random.choice(range(len(test_data))) # select a random conversation
print(idx)
dialogue = test_data['chat'][idx]
print(dialogue)

19
User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer t (1 ≤ t ≤

In [ ]:
# now take only the 'input part' and the 'output part'
# parse string
test_input, test_output, end = dialogue.split('<|endoftext|>')

# add eos token at the end of test_input and test_output
test_input = test_input + tokenizer_gpt2.eos_token
test_output = test_output + tokenizer_gpt2.eos_token

print('User input: \n', test_input)
print('####')
print('Correct solution output: \n', test_output)

User input: 
 User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line contains a single integer

In [ ]:
!pip install pipeline transformers evaluate
from transformers import pipeline
import evaluate
pipe_finetuned = pipeline("text-generation", model= gpt2, tokenizer=tokenizer_gpt2)

In [ ]:
gen_output = pipe_finetuned(test_input, max_new_tokens=256)
print("Generated text after the fine-tuning: \n", gen_output[0]["generated_text"])

Generated text after the fine-tuning: 
 User: Bob is playing with 6-sided dice. A net of such standard cube is shown below.

<image>

He has an unlimited supply of these dice and wants to build a tower by stacking multiple dice on top of each other, while choosing the orientation of each dice. Then he counts the number of visible pips on the faces of the dice.

For example, the number of visible pips on the tower below is 29 — the number visible on the top is 1, from the south 5 and 3, from the west 4 and 2, from the north 2 and 4 and from the east 3 and 5.

<image>

The one at the bottom and the two sixes by which the dice are touching are not visible, so they are not counted towards total.

Bob also has t favourite integers x_i, and for every such integer his goal is to build such a tower that the number of visible pips is exactly x_i. For each of Bob's favourite integers determine whether it is possible to build a tower that has exactly that many visible pips.

Input

The first line

The generated text after the fine-tuning includes now the python code. The code is still not correct, especially from a syntactical point of view, but the model is able to maintain the espected format of the response ('user:' and 'assistant:')

#### Evaluation Metrics

It was decided to consider the metrics Perplexity and BLEU.

**Perplexity**

In [ ]:
# get inputs from test_data
test_input = [dialogue.split('<|endoftext|>')[0] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

# get outputs from test data
test_output = [dialogue.split('<|endoftext|>')[1] + tokenizer_gpt2.eos_token for dialogue in test_data['chat'][:3]]

In [ ]:
perplexity_metric = evaluate.load("perplexity", module_type="metric")
perplexity = perplexity_metric.compute(
    predictions= test_input,
    add_start_token=False,
    model_id= gpt2_finetune_path
)
print("Perplexity post fine-tuning:", perplexity)

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity post fine-tuning: {'perplexities': [1.080601692199707, 1.1230411529541016, 1.0684187412261963], 'mean_perplexity': np.float64(1.0906871954600017)}


The perplexity calculated before the fine-tuning was [653.0138549804688, 570.7035522460938, 424.50201416015625] with mean_perplexity: np.float64(549.4064737955729)

Now we can notice how the values for the perplexity are much lower, similar to those achieved by the gpt2-small model, indicating how the fine-tuning process managed to improve the performance.

**BLEU**

BLEU focuses on precision by counting matching n-grams.

In [ ]:
# BLEU Score
bleu_metric = evaluate.load("bleu")
bleu_results = bleu_metric.compute(
    predictions= [pipe_finetuned(test_input[0], max_new_tokens=256)[0]['generated_text']], 
    references= [test_output[0]]  
)
print("BLEU Score:", bleu_results)

BLEU Score: {'bleu': 0.05315658078526119, 'precisions': [0.3118081180811808, 0.10351201478743069, 0.03333333333333333, 0.0074211502782931356], 'brevity_penalty': 1.0, 'length_ratio': 1.7828947368421053, 'translation_length': 542, 'reference_length': 304}


The Bleu score, even if higher than the scores achieved by the T5 models, are still low, indicating that the generated text is still far from the reference text. 

# Try some LLMs on the task

The notebook tests how different pre-trained language models perform code generation from a textual problem description. Three prompting techniques are explored:

* Zero-shot: The model receives only the problem statement.

* One-shot: One example of a solved problem is provided.

* Few-shot: Multiple examples are given before asking the model to solve a new problem.

Three different LLMs are compared across these settings:


* TinyLlama-1.1B: 1.1B parameters (https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)
* deepseek-coder-1.3b-base: 1.3B parameters (https://huggingface.co/deepseek-ai/deepseek-coder-1.3b-base)
* Mistral-7B-Instruct-v0.1: 7B parameters (https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1)



## Imports

This section installs and imports all necessary libraries for the notebook

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=cd3991a622edd739af4a9ae10905bdea7e64d0c09a9b9a6bc7ac3fc56813fa84
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 25.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.6.0
    Uninstalling accelerate-1.6.0:
      Successfully uninstalled accelerate-1.6.0


In [ ]:
!pip install -q -U langchain

In [ ]:
!pip install -q langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import torch
import pandas as pd
from transformers import pipeline
import sacrebleu
from rouge_score import rouge_scorer
import json


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

## Load Dataset

Load the dataset that has already been preprocessed

In [ ]:
drive.mount('/content/drive')

path = 'Colab Notebooks/NLP/NLP_Project'

os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

Mounted at /content/drive


'/content/drive/.shortcut-targets-by-id/17WgJO1gfIBADpYX2jVdb41q7HCbwWcOU/NLP_Project'

In [ ]:
df = pd.read_csv('final_ds.csv')
print(df.head())

                                 problem_description solution_id  \
0  Xenia has a set of weights and pan scales. Eac...         0_0   
1  Xenia has a set of weights and pan scales. Eac...         0_2   
2  Xenia has a set of weights and pan scales. Eac...         0_4   
3  Xenia has a set of weights and pan scales. Eac...         0_6   
4  Xenia has a set of weights and pan scales. Eac...         0_8   

                                       solution_code  \
0  __author__ = 'ratnesh.mishra'\n\nweights = map...   
1  import sys\nsys.setrecursionlimit (1000000)\n\...   
2  import sys\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...   
3  MOD = 10**9 + 7\nI = lambda:list(map(int,input...   
4  to_print = []\ndef dfs(d, ini, s, depth, m):\n...   

               problem_name time_complexity_inferred space_complexity_inferred  
0  339_C. Xenia and Weights                     O(1)                   O(n**2)  
1  339_C. Xenia and Weights                     O(1)                      O(1)  
2  339_C. X

In [ ]:
df.shape

(244876, 6)

## Try TinyLlama on the task

The TinyLlama-1.1B-Chat-v1.0 is a compact open-source language model with 1.1 billion parameters, trained for chat and code generation tasks. This version is fine-tuned for multi-turn conversations and instruction-following behavior.

Here we initializes a Hugging Face pipeline for text generation, the pre-trained model.

In [ ]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


This function creates a list of chat-style messages to prompt a language model for code generation, using zero-shot, one-shot or few-shot prompting strategies.

The returned messages list is formatted with "system", "user", and "assistant" roles.

In [ ]:
def build_messages(df, index, mode='zero-shot', num_few_shot=2, random_state=0):
    problem = clean_problem_description(df.loc[index, 'problem_description'])
    messages = [
        {
            "role": "system",
            "content": "You are an assistent who helps solve programming problems by writing the corresponding Python code. Please respond with the code only, without explanation."
        }
    ]

    if mode == 'zero-shot':
        messages.append({
            "role": "user",
            "content": f"Can you please solve the problem below by writing the corresponding Python code.\n\n"
                       f"### Problem:\n{problem}\n### Your Solution:"
        })

    elif mode == 'one-shot':
        example_idx = df.drop(index).sample(1, random_state=random_state).index[0]
        example_problem = clean_problem_description(df.loc[example_idx, 'problem_description'])
        example_solution = df.loc[example_idx, 'solution_code']

        messages.append({
            "role": "user",
            "content": f"### Problem:\n{example_problem}\n### Solution Expected:"
        })
        messages.append({
            "role": "assistant",
            "content": f"{example_solution}"
        })
        messages.append({
            "role": "user",
            "content": (
                "Now please solve the problem below by writing the corresponding Python code.\n\n"
                f"### Problem:\n{problem}\n### Your Solution:"
            )
        })

    elif mode == 'few-shot':
        example_indices = df.drop(index).sample(num_few_shot, random_state=random_state).index

        for i in example_indices:
            example_problem = clean_problem_description(df.loc[i, 'problem_description'])
            example_solution = df.loc[i, 'solution_code']
            messages.append({
                "role": "user",
                "content": f"### Problem:\n{example_problem}\n### Solution Expected:"
            })
            messages.append({
                "role": "assistant",
                "content": f"{example_solution}"
            })

        messages.append({
            "role": "user",
            "content": (
                "Now please solve the problem below by writing the corresponding Python code.\n\n"
                f"### Problem:\n{problem}\n### Your Solution:"
            )
        })

    return messages


This function uses the chat-formatted messages to build a prompt and pass it to the language model for code generation. The following generation parameters are used:

* max_new_tokens=256: limits the number of new tokens the model can generate
* do_sample=True: enables sampling, which allows the model to produce more diverse and creative outputs instead of always choosing the most likely next token.
* temperature=0.7: controls the randomness of the predictions.
* top_k=50: restricts sampling to the top 50 most probable tokens at each step.
* top_p=0.95: enables nucleus sampling, where the model considers only the smallest possible set of tokens whose cumulative probability is at least 95%.

The model returns the generated Python solution as plain text.

In [ ]:
def generate_solution(messages, max_new_tokens=256):
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    return outputs[0]["generated_text"]

This function evaluates the similarity between a generated solution and a reference solution:
* BLEU: Uses sacrebleu to compute n-gram precision.
* ROUGE-1 / ROUGE-L: Measures recall-based overlap (unigrams and longest common subsequence).
* Exact Match: Checks if the strings are exactly equal after stripping whitespace.

Returns a dictionary with all four metrics.

In [ ]:
def evaluate_metrics(reference, hypothesis):
    bleu_score = sacrebleu.corpus_bleu([hypothesis], [[reference]]).score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(reference, hypothesis)
    exact = int(reference.strip() == hypothesis.strip())
    return {
        'bleu': bleu_score,
        'rouge1': rouge_scores['rouge1'].fmeasure,
        'rougeL': rouge_scores['rougeL'].fmeasure,
        'exact_match': exact
    }

This function is used to remove the examples inputs and outputs of the problem description, so the one and few shot prompts are less confusing for the model

In [ ]:
def clean_problem_description(text: str) -> str:
    lines = text.strip().splitlines()
    result = []
    for line in lines:
        if line.strip().lower().startswith("examples"):
            break
        result.append(line)
    return "\n".join(result).strip()


Here we get a random sample of the dataset so we can test the three prompt types

In [ ]:
example = df.sample(n=1, random_state=40)

In [ ]:
n = example.index[0]

In [ ]:
n

np.int64(15399)

In [ ]:
example_text = df.iloc[n]

In [ ]:
clean_text = clean_problem_description(df.loc[n, 'problem_description'])

In [ ]:
clean_text

'There are H rows and W columns of white square cells.\n\nYou will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.\n\nHow many white cells will remain?\n\nIt can be proved that this count does not depend on what rows and columns are chosen.\n\nConstraints\n\n* All values in input are integers.\n* 1 \\leq H, W \\leq 20\n* 1 \\leq h \\leq H\n* 1 \\leq w \\leq W\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nH W\nh w\n\n\nOutput\n\nPrint the number of white cells that will remain.'

In [ ]:
print(example_text['problem_description'])

There are H rows and W columns of white square cells.

You will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.

How many white cells will remain?

It can be proved that this count does not depend on what rows and columns are chosen.

Constraints

* All values in input are integers.
* 1 \leq H, W \leq 20
* 1 \leq h \leq H
* 1 \leq w \leq W

Input

Input is given from Standard Input in the following format:


H W
h w


Output

Print the number of white cells that will remain.

Examples

Input

3 2
2 1


Output

1


Input

5 5
2 3


Output

6


Input

2 4
2 4


Output

0


Here we can the the description of the problem we are going to use to test the prompts. It receives four numbers a, b, c and d as inputs and should output (a-c)*(b-d) as the number of remaining white cells.

In [ ]:
example_text['solution_code']

'a,b=map(int,input().split())\nc,d=map(int,input().split())\nprint((a-c)*(b-d))'

First we need to build the zero shot prompt using the build_messages function and specifying the zero-shot approach

In [ ]:
zero_shot_messages = build_messages(df, example.index[0], mode='zero-shot')
print(zero_shot_messages)

[{'role': 'system', 'content': 'You are an assistent who helps solve programming problems by writing the corresponding Python code. Please respond with the code only, without explanation.'}, {'role': 'user', 'content': 'Can you please solve the problem below by writing the corresponding Python code.\n\n### Problem:\nThere are H rows and W columns of white square cells.\n\nYou will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.\n\nHow many white cells will remain?\n\nIt can be proved that this count does not depend on what rows and columns are chosen.\n\nConstraints\n\n* All values in input are integers.\n* 1 \\leq H, W \\leq 20\n* 1 \\leq h \\leq H\n* 1 \\leq w \\leq W\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nH W\nh w\n\n\nOutput\n\nPrint the number of white cells that will remain.\n### Your Solution:'}]


Now we pass this prompt to the system and print the generated output

In [ ]:
zero_shot_generated = generate_solution(zero_shot_messages)
print(zero_shot_generated)

<|system|>
You are an assistent who helps solve programming problems by writing the corresponding Python code. Please respond with the code only, without explanation.</s>
<|user|>
Can you please solve the problem below by writing the corresponding Python code.

### Problem:
There are H rows and W columns of white square cells.

You will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.

How many white cells will remain?

It can be proved that this count does not depend on what rows and columns are chosen.

Constraints

* All values in input are integers.
* 1 \leq H, W \leq 20
* 1 \leq h \leq H
* 1 \leq w \leq W

Input

Input is given from Standard Input in the following format:


H W
h w


Output

Print the number of white cells that will remain.
### Your Solution:</s>
<|assistant|>
Here's the Python code for the problem:

```python
import sys

# Check input format
if len(sys.argv)!= 2:
    print("Usage: python3 solution.py <input

The code generated by the model for this problem was:

In [ ]:
import sys

# Check input format
if len(sys.argv)!= 2:
    print("Usage: python3 solution.py <input_file_name>")
    sys.exit(1)

try:
    input_file = open(sys.argv[1], "r")
except FileNotFoundError:
    print("Input file not found!")
    sys.exit(1)

try:
    n = int(input_file.readline())
    h = int(input_file.readline())
    w = int(input_file.readline())
    input_file.close()
except ValueError:
    print("Invalid input format!")
    sys.exit(1)

# Check if all values are integers
while h > 0 or w > 0:
    try:
        h = int(input_file.readline())
    except ValueError:
        print("Invalid input format!")
        sys.exit(1)
    try:
        w = int(input_file.read

The generated code attempts to read input from a file using sys.argv, but it is incorrect for the proposed problem. It closes the file before finishing reading, and contains incomplete and invalid chunks. And it does not solve the problem correctly or print the expected output.

However, we can see that the model is already generating Python code with a reasonable structure, including basic input handling, error checking, and use of standard libraries.

Now we see if the model performs better with the one-shot prompt approach. In this cell we generate the one shot prompt using also a random example.

In [ ]:
one_shot_messages = build_messages(df, example.index[0], mode='one-shot')
print(one_shot_messages)

[{'role': 'system', 'content': 'You are an assistent who helps solve programming problems by writing the corresponding Python code. Please respond with the code only, without explanation.'}, {'role': 'user', 'content': '### Problem:\nYou are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.\n\nAt least how many operations are necessary to satisfy the following conditions?\n\n* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.\n* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.\n\nConstraints\n\n* 2 ≤ n ≤ 10^5\n* |a_i| ≤ 10^9\n* Each a_i is an integer.\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nn\na_1 a_2 ... a_n\n\n\nOutput\n\nPrint the minimum necessary count of operations.\n### 

And now we generate the output for this prompt

In [ ]:
one_shot_generated = generate_solution(one_shot_messages)
print(one_shot_generated)

<|system|>
You are an assistent who helps solve programming problems by writing the corresponding Python code. Please respond with the code only, without explanation.</s>
<|user|>
### Problem:
You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.
### Solution Expected:</s>
<|assistant|>
N = int(input())
a = list(map

Now the code generated was:

In [ ]:
N = int(input())
h = int(input())
w = int(input())

if h * w <= 0:
    print(1)
else:
    cells_remaining = 0
    for row in range(h):
        for col in range(w):
            if row * w + col <= h * w:
                cells_remaining += 1
    print(cells_remaining)

Which still has a well structured python code but logically doesn't solve the proposed problem. Now we can also see that after the code the model tried to explain the logic behind it.

The last approach if the few-shot, giving the model two examples instead of just one

In [ ]:
few_shot_messages = build_messages(df, example.index[0], mode='few-shot')
print(few_shot_messages)

[{'role': 'system', 'content': 'You are an assistent who helps solve programming problems by writing the corresponding Python code. Please respond with the code only, without explanation.'}, {'role': 'user', 'content': '### Problem:\nYou are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.\n\nAt least how many operations are necessary to satisfy the following conditions?\n\n* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.\n* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.\n\nConstraints\n\n* 2 ≤ n ≤ 10^5\n* |a_i| ≤ 10^9\n* Each a_i is an integer.\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nn\na_1 a_2 ... a_n\n\n\nOutput\n\nPrint the minimum necessary count of operations.\n### 

In [ ]:
few_shot_generated = generate_solution(few_shot_messages)
print(few_shot_generated)

<|system|>
You are an assistent who helps solve programming problems by writing the corresponding Python code. Please respond with the code only, without explanation.</s>
<|user|>
### Problem:
You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.
### Solution Expected:</s>
<|assistant|>
N = int(input())
a = list(map

The code generated this time was:

In [ ]:
N=int(input())
h=int(input()[0])
w=int(input()[1])

# Find the number of white cells in H rows and W columns
rows_num=0
for I in range(h):
    for j in range(w):
        if input()[i*w+j] == 'w':
            rows_num+=1

# Calculate the number of white cells in the remaining H rows and W columns
remaining_rows_num=0
for I in range(h-1, -1, -1):
    for j in range(w):
        if input()[i*w+j] == 'w':
            remaining_rows_num+=1

# Calculate the number of white cells remaining in H rows and W columns
remaining_cells_num=rows_num-remaining_rows_num

print(remaining_cells_num)

The few-shot output better aligns with the grid structure of the problem and introduces more descriptive variable names. However, it still misinterprets the input format and fails to implement the correct logic.

Now we will run the three prompting approaches with a few samples to get some metrics

In [ ]:
results = {'zero-shot': [], 'one-shot': [], 'few-shot': []}

In [ ]:
sample_indices = df.sample(5, random_state=0).index

In [ ]:
saved_results = []

for mode in ['zero-shot', 'one-shot', 'few-shot']:
    for idx in sample_indices:
        messages = build_messages(df, idx, mode=mode, num_few_shot=3)
        generated = generate_solution(messages, max_new_tokens=256)

        expected = df.loc[idx, 'solution_code']

        metrics = evaluate_metrics(expected, generated)

        print(f"Mode: {mode}, Example {idx}")
        #print("Generated:\n", generated)
        #print("Expected:\n", expected)
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        print("\n")

        results[mode].append(metrics)

        saved_results.append({
            'mode': mode,
            'index': idx,
            'problem_description': df.loc[idx, 'problem_description'],
            'expected_solution': expected,
            'generated_solution': generated,
            'bleu': metrics['bleu'],
            'rouge1': metrics['rouge1'],
            'rougeL': metrics['rougeL'],
            'exact_match': metrics['exact_match']
        })


Mode: zero-shot, Example 219235
bleu: 1.1744
rouge1: 0.0337
rougeL: 0.0337
exact_match: 0.0000


Mode: zero-shot, Example 116625
bleu: 4.5137
rouge1: 0.1049
rougeL: 0.0918
exact_match: 0.0000


Mode: zero-shot, Example 161440
bleu: 1.8577
rouge1: 0.1154
rougeL: 0.0726
exact_match: 0.0000


Mode: zero-shot, Example 177012
bleu: 8.3524
rouge1: 0.2725
rougeL: 0.1755
exact_match: 0.0000


Mode: zero-shot, Example 26857
bleu: 7.2144
rouge1: 0.2652
rougeL: 0.1215
exact_match: 0.0000


Mode: one-shot, Example 219235
bleu: 1.2996
rouge1: 0.0182
rougeL: 0.0182
exact_match: 0.0000




You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Mode: one-shot, Example 116625
bleu: 2.4819
rouge1: 0.0784
rougeL: 0.0610
exact_match: 0.0000


Mode: one-shot, Example 161440
bleu: 1.9589
rouge1: 0.1223
rougeL: 0.0699
exact_match: 0.0000


Mode: one-shot, Example 177012
bleu: 5.9250
rouge1: 0.2358
rougeL: 0.1440
exact_match: 0.0000


Mode: one-shot, Example 26857
bleu: 4.6816
rouge1: 0.2055
rougeL: 0.1054
exact_match: 0.0000


Mode: few-shot, Example 219235
bleu: 0.6755
rouge1: 0.0095
rougeL: 0.0095
exact_match: 0.0000




This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Mode: few-shot, Example 116625
bleu: 1.6039
rouge1: 0.0285
rougeL: 0.0268
exact_match: 0.0000


Mode: few-shot, Example 161440
bleu: 1.8570
rouge1: 0.0734
rougeL: 0.0500
exact_match: 0.0000


Mode: few-shot, Example 177012
bleu: 3.7599
rouge1: 0.1353
rougeL: 0.0844
exact_match: 0.0000


Mode: few-shot, Example 26857
bleu: 4.0815
rouge1: 0.1560
rougeL: 0.0705
exact_match: 0.0000




In [ ]:
for mode in results:
    avg_metrics = {}
    for metric in results[mode][0].keys():
        avg_metrics[metric] = sum([ex[metric] for ex in results[mode]]) / len(results[mode])

    print(f"\nAverage metrics for {mode}:")
    for metric, value in avg_metrics.items():
        print(f"{metric}: {value:.4f}")


Average metrics for zero-shot:
bleu: 4.6225
rouge1: 0.1583
rougeL: 0.0990
exact_match: 0.0000

Average metrics for one-shot:
bleu: 3.2694
rouge1: 0.1321
rougeL: 0.0797
exact_match: 0.0000

Average metrics for few-shot:
bleu: 2.3955
rouge1: 0.0805
rougeL: 0.0482
exact_match: 0.0000


These results show that the zero-shot setting outperforms one-shot and few-shot in all metrics, including BLEU, ROUGE-1, ROUGE-L, and exact match (which is zero across the board). This suggests that providing examples (one-shot or few-shot) did not help the model improve code generation quality for this task, and may have introduced noise or confusion, leading to worse performance.
This can be caused by poorly formatted prompts or overly long or complex example prompts

In [ ]:
df_results = pd.DataFrame(saved_results)
df_results.to_csv('generated_codes.csv', index=False)

In [ ]:
with open('generated_codes.json', 'w', encoding='utf-8') as f:
    json.dump(saved_results, f, ensure_ascii=False, indent=4)

## Try deepseek-coder on the task

DeepSeek-Coder-1.3B-Base is a 1.3 billion parameter language model specialized in code generation and understanding. It is trained on a diverse set of programming languages and designed to assist with tasks such as code completion, bug fixing, and problem-solving by generating syntactically correct and contextually relevant code snippets.

We start by loading the model and the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True).cuda()

tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

This function is used to build the prompts in each of the approaches. It tries to give a well structured prompt with clearly separated examples

In [ ]:
def build_prompt_deepseek(df, index, mode='zero-shot', num_few_shot=3, random_state=0):
    problem = clean_problem_description(df.loc[index, 'problem_description'])
    if mode == 'zero-shot':
        prompt = f"# Task: {problem}\n# Solution:\n"

    elif mode == 'one-shot':
        example_idx = df.drop(index).sample(1, random_state=random_state).index[0]
        example_problem = clean_problem_description(df.loc[example_idx, 'problem_description'])
        example_solution = df.loc[example_idx, 'solution_code']
        prompt = (
            f"# === Example 1 ===\n"
            f"# Task: {example_problem}\n# Solution:\n{example_solution}\n\n"
            f"# Task: {problem}\n# Solution:\n"
        )

    elif mode == 'few-shot':
        example_indices = df.drop(index).sample(num_few_shot, random_state=random_state).index
        prompt = ""
        for j, i in enumerate(example_indices):
            example_problem = clean_problem_description(df.loc[i, 'problem_description'])
            example_solution = df.loc[i, 'solution_code']
            prompt += (
                f"# === Example {j+1} ===\n"
                f"# Task: {example_problem}\n"
                f"# Solution:\n{example_solution}\n\n"
            )
        prompt += f"# Task: {problem}\n# Solution:\n"

    return prompt


This function is used to generate the model output using the prompt build before and returns the string containing the output

In [ ]:
def generate_solution_deepseek(prompt, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

We will use the same problem example as before for demonstration

In [ ]:
n

np.int64(14480)

In [ ]:
example

,problem_description,solution_id,solution_code,problem_name,time_complexity_inferred,space_complexity_inferred
14480,There are H rows and W columns of white square...,190_3,"a,b=map(int,input().split())\nc,d=map(int,inpu...",p03101 AtCoder Beginner Contest 121 - White Cells,O(1),O(1)


In [ ]:
zero_shot_prompt = build_prompt_deepseek(df, example.index[0], mode='zero-shot')
print(zero_shot_prompt)

# Task: There are H rows and W columns of white square cells.

You will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.

How many white cells will remain?

It can be proved that this count does not depend on what rows and columns are chosen.

Constraints

* All values in input are integers.
* 1 \leq H, W \leq 20
* 1 \leq h \leq H
* 1 \leq w \leq W

Input

Input is given from Standard Input in the following format:


H W
h w


Output

Print the number of white cells that will remain.
# Solution:



In [ ]:
zero_shot_generated = generate_solution_deepseek(zero_shot_prompt)
print(zero_shot_generated)

# Task: There are H rows and W columns of white square cells.

You will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.

How many white cells will remain?

It can be proved that this count does not depend on what rows and columns are chosen.

Constraints

* All values in input are integers.
* 1 \leq H, W \leq 20
* 1 \leq h \leq H
* 1 \leq w \leq W

Input

Input is given from Standard Input in the following format:


H W
h w


Output

Print the number of white cells that will remain.
# Solution:


def white_cells(H, W, h, w):
    # Write your code here
    return (H - h) * (W - w)


if __name__ == '__main__':
    H, W = map(int, input().split())
    h, w = map(int, input().split())
    print(white_cells(H, W, h, w))


The code generated by the zero-shot approach was:

In [ ]:
def white_cells(H, W, h, w):
    # Write your code here
    return (H - h) * (W - w)


if __name__ == '__main__':
    H, W = map(int, input().split())
    h, w = map(int, input().split())
    print(white_cells(H, W, h, w))

Which correctly solves the problem proposed.

TinyLlama, being a general-purpose language model with fewer parameters and less specialized training on code, may struggle with understanding problem constraints and generating correct, concise solutions. In contrast, DeepSeek-Coder is specifically trained on diverse programming tasks and code generation, enabling it to better grasp problem requirements and produce accurate, efficient code.

In [ ]:
one_shot_prompt = build_prompt_deepseek(df, example.index[0], mode='one-shot')
print(one_shot_prompt)

# === Example 1 ===
# Task: You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.
# Solution:
N = int(input())
a = list(map(int, input().split()))

ans1 = 0
s = 0
flg = 1
for ai in a:
    s += ai
    if s * flg <= 0:
        ans1 += abs(s) + 1
        s = flg
    flg *= -1

ans2 = 0
s = 0
flg = -1
for ai in a:
    s

In [ ]:
one_shot_generated = generate_solution_deepseek(one_shot_prompt)
print(one_shot_generated)

# === Example 1 ===
# Task: You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.
# Solution:
N = int(input())
a = list(map(int, input().split()))

ans1 = 0
s = 0
flg = 1
for ai in a:
    s += ai
    if s * flg <= 0:
        ans1 += abs(s) + 1
        s = flg
    flg *= -1

ans2 = 0
s = 0
flg = -1
for ai in a:
    s

The solution generated by the one-shot prompt was:

In [ ]:
# Solution:
H, W = map(int, input().split())
h, w = map(int, input().split())
print((H - h + 1) * (W - w + 1) - len(set(input().split())))

The generated solution overcomplicates the problem by using an incorrect formula and expecting extra input that is not required. This likely happened because the one-shot example used in the prompt added noise or confusion, leading the model to misinterpret the problem’s requirements and produce a more complex, incorrect solution than necessary.

In [ ]:
few_shot_prompt = build_prompt_deepseek(df, example.index[0], mode='few-shot')
print(few_shot_prompt)

# === Example 1 ===
# Task: You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.
# Solution:
N = int(input())
a = list(map(int, input().split()))

ans1 = 0
s = 0
flg = 1
for ai in a:
    s += ai
    if s * flg <= 0:
        ans1 += abs(s) + 1
        s = flg
    flg *= -1

ans2 = 0
s = 0
flg = -1
for ai in a:
    s

In [ ]:
few_shot_generated = generate_solution_deepseek(few_shot_prompt)
print(few_shot_generated)

# === Example 1 ===
# Task: You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.
# Solution:
N = int(input())
a = list(map(int, input().split()))

ans1 = 0
s = 0
flg = 1
for ai in a:
    s += ai
    if s * flg <= 0:
        ans1 += abs(s) + 1
        s = flg
    flg *= -1

ans2 = 0
s = 0
flg = -1
for ai in a:
    s

The code generated by the model was:

In [ ]:
H,W=map(int,input().split())
h,w=map(int,input().split())
print((H-h)*(W-w))

Which correctly solves the problem, and actually is almost the same one as the one provided in the dataset:

In [ ]:
a,b=map(int,input().split())
c,d=map(int,input().split())
print((a-c)*(b-d))

But we can see that after the model continued generating new examples and solution codes, like it was continuing the prompt. Long or confusing examples can cause the model to not recognize where to end.

In [ ]:
results_deepseek = {'zero-shot': [], 'one-shot': [], 'few-shot': []}

In [ ]:
saved_results_deepseek = []

for mode in ['zero-shot', 'one-shot', 'few-shot']:
    for idx in sample_indices:
        prompt = build_prompt_deepseek(df, idx, mode=mode, num_few_shot=3)
        generated = generate_solution_deepseek(prompt, max_new_tokens=256)

        expected = df.loc[idx, 'solution_code']

        metrics = evaluate_metrics(expected, generated)

        print(f"Mode: {mode}, Example {idx}")
        #print("Generated:\n", generated)
        #print("Expected:\n", expected)
        for metric, value in metrics.items():
            print(f"{metric}: {value:.4f}")
        print("\n")

        results_deepseek[mode].append(metrics)

        saved_results_deepseek.append({
            'mode': mode,
            'index': idx,
            'problem_description': df.loc[idx, 'problem_description'],
            'expected_solution': expected,
            'generated_solution': generated,
            'bleu': metrics['bleu'],
            'rouge1': metrics['rouge1'],
            'rougeL': metrics['rougeL'],
            'exact_match': metrics['exact_match']
        })


Mode: zero-shot, Example 219235
bleu: 3.1388
rouge1: 0.0556
rougeL: 0.0556
exact_match: 0.0000


Mode: zero-shot, Example 116625
bleu: 2.3394
rouge1: 0.1863
rougeL: 0.1242
exact_match: 0.0000


Mode: zero-shot, Example 161440
bleu: 8.3381
rouge1: 0.1355
rougeL: 0.1030
exact_match: 0.0000


Mode: zero-shot, Example 177012
bleu: 11.4481
rouge1: 0.2696
rougeL: 0.1569
exact_match: 0.0000


Mode: zero-shot, Example 26857
bleu: 14.0492
rouge1: 0.3170
rougeL: 0.1445
exact_match: 0.0000


Mode: one-shot, Example 219235
bleu: 1.2779
rouge1: 0.0173
rougeL: 0.0173
exact_match: 0.0000


Mode: one-shot, Example 116625
bleu: 2.8054
rouge1: 0.0677
rougeL: 0.0558
exact_match: 0.0000


Mode: one-shot, Example 161440
bleu: 5.9441
rouge1: 0.1477
rougeL: 0.0937
exact_match: 0.0000


Mode: one-shot, Example 177012
bleu: 11.4539
rouge1: 0.2831
rougeL: 0.1788
exact_match: 0.0000


Mode: one-shot, Example 26857
bleu: 4.1924
rouge1: 0.1988
rougeL: 0.0906
exact_match: 0.0000


Mode: few-shot, Example 219235
ble

Now we will run the three prompting approaches with a few samples to get some metrics

In [ ]:
for mode in results_deepseek:
    avg_metrics = {}
    for metric in results_deepseek[mode][0].keys():
        avg_metrics[metric] = sum([ex[metric] for ex in results_deepseek[mode]]) / len(results_deepseek[mode])

    print(f"\nAverage metrics for {mode}:")
    for metric, value in avg_metrics.items():
        print(f"{metric}: {value:.4f}")


Average metrics for zero-shot:
bleu: 7.8627
rouge1: 0.1928
rougeL: 0.1168
exact_match: 0.0000

Average metrics for one-shot:
bleu: 5.1347
rouge1: 0.1429
rougeL: 0.0872
exact_match: 0.0000

Average metrics for few-shot:
bleu: 2.6901
rouge1: 0.0828
rougeL: 0.0537
exact_match: 0.0000


Again we have worse metrics for the few-shot approach than to the other two. This can also be caused by the model continuing the generation after the solution, like the example above

In [ ]:
df_results_deepseek = pd.DataFrame(saved_results_deepseek)
df_results_deepseek.to_csv('generated_codes_deepseek.csv', index=False)

In [ ]:
with open('generated_codes_deepseek.json', 'w', encoding='utf-8') as f:
    json.dump(saved_results_deepseek, f, ensure_ascii=False, indent=4)

## Try Mistral-7B-Instruct-v0.1 on the task

Mistral-7B-Instruct-v0.1 is a 7-billion parameter language model designed for instruction-following tasks. It offers strong performance in natural language understanding and generation. Due to its large size, we will use quantization techniques to reduce memory usage and enable efficient inference on limited hardware.

This code configures 4-bit quantization for loading a model with the BitsAndBytes library. It specifies that the model weights should be loaded in 4-bit precision to reduce memory usage, using the "nf4" quantization type and double quantization for better accuracy.

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

To access the model we need to login to HuggingFace

In [ ]:
from huggingface_hub import login
from google.colab import userdata
#token = userdata.get('HF_TOKEN')
login(token=token)

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

This code loads the Mistral-7B-Instruct-v0.1 model with 4-bit quantization enabled. The device_map="auto" argument automatically assigns model layers to available hardware devices. The tokenizer is also loaded to preprocess input text for the model.

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Now we create a text-generation pipeline using the 4-bit quantized Mistral model and its tokenizer. We configure generation parameters like maximum output length, sampling with top-k filtering, and device placement. The pipeline is then wrapped in a HuggingFacePipeline object for easy integration with other tools.

In [ ]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        truncation=True,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline_inst)

Device set to use cuda:0
<ipython-input-64-81eff7bed817>:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline_inst)


This function builds a prompt and generates a Python code solution for a programming problem based on the given mode: zero-shot, one-shot, or few-shot.  The prompt is wrapped in a PromptTemplate and passed to an LLM chain (llm_chain) to produce the final code-only response.

In [ ]:
import random

template = """You are a helpful assistant that generates Python code to solve the given problem.
You should respond only with the code, without explanation.

### Problem
{question}

### Solution
"""

def generate_response(df, index, mode='zero-shot', num_few_shot=3, random_state=0):
    random.seed(random_state)

    question = clean_problem_description(df.loc[index, 'problem_description'])

    if mode == 'zero-shot':
        prompt = template.format(question=question)

    elif mode == 'one-shot':
        example_idx = df.drop(index).sample(1, random_state=random_state).index[0]
        example_problem = clean_problem_description(df.loc[example_idx, 'problem_description'])
        example_solution = df.loc[example_idx, 'solution_code']

        prompt = (
            "You are a helpful assistant that generates Python code to solve the given problem.\n"
            "You should respond only with the code, without explanation.\n\n"
            "### Problem\n"
            f"{example_problem}\n\n"
            "### Solution\n"
            f"{example_solution.strip()}\n"
            "---\n"
            "### Problem\n"
            f"{question}\n\n"
            "### Solution\n"
        )

    elif mode == 'few-shot':
        example_indices = df.drop(index).sample(num_few_shot, random_state=random_state).index
        prompt = (
            "You are a helpful assistant that generates Python code to solve the given problem.\n"
            "You should respond only with the code, without explanation.\n\n"
        )
        for i in example_indices:
            example_problem = clean_problem_description(df.loc[i, 'problem_description'])
            example_solution = df.loc[i, 'solution_code']
            prompt += (
                "### Problem\n"
                f"{example_problem}\n\n"
                "### Solution\n"
                f"{example_solution.strip()}\n"
                "---\n"
            )
        prompt += (
            "### Problem\n"
            f"{question}\n\n"
            "### Solution\n"
        )

    else:
        raise ValueError("Invalid mode. Choose between 'zero-shot', 'one-shot', or 'few-shot'.")

    prompt_template = PromptTemplate(template=prompt, input_variables=["question"])
    llm_chain = LLMChain(prompt=prompt_template, llm=llm)

    response = llm_chain.run({"question": question})
    return response


We will also use the same example from before for the demonstration

In [ ]:
example.index[0]

np.int64(14480)

In [ ]:
df.loc[example.index[0], 'problem_description']


'There are H rows and W columns of white square cells.\n\nYou will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.\n\nHow many white cells will remain?\n\nIt can be proved that this count does not depend on what rows and columns are chosen.\n\nConstraints\n\n* All values in input are integers.\n* 1 \\leq H, W \\leq 20\n* 1 \\leq h \\leq H\n* 1 \\leq w \\leq W\n\nInput\n\nInput is given from Standard Input in the following format:\n\n\nH W\nh w\n\n\nOutput\n\nPrint the number of white cells that will remain.\n\nExamples\n\nInput\n\n3 2\n2 1\n\n\nOutput\n\n1\n\n\nInput\n\n5 5\n2 3\n\n\nOutput\n\n6\n\n\nInput\n\n2 4\n2 4\n\n\nOutput\n\n0'

In [ ]:
output = generate_response(df, example.index[0], mode='zero-shot')
print(output)

<ipython-input-65-6cdfcba6b64f>:61: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt_template, llm=llm)
<ipython-input-65-6cdfcba6b64f>:63: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({"question": question})


You are a helpful assistant that generates Python code to solve the given problem.
You should respond only with the code, without explanation.
Problem description:
There are H rows and W columns of white square cells.

You will choose h of the rows and w of the columns, and paint all of the cells contained in those rows or columns.

How many white cells will remain?

It can be proved that this count does not depend on what rows and columns are chosen.

Constraints

* All values in input are integers.
* 1 \leq H, W \leq 20
* 1 \leq h \leq H
* 1 \leq w \leq W

Input

Input is given from Standard Input in the following format:


H W
h w


Output

Print the number of white cells that will remain.
Solution (Python code):

from itertools import combinations

n, m = map(int, input().split())
h, w = map(int, input().split())

combs = [(i, j) for i, j in combinations(range(n), w)]

white_cells = sum([1 for _, j in combs if j < m])
print(white_cells)

Note: This code uses Python 3.7 and above. I

The code generated by the zero shot approach was:

In [ ]:
from itertools import combinations

n, m = map(int, input().split())
h, w = map(int, input().split())

combs = [(i, j) for i, j in combinations(range(n), w)]

white_cells = sum([1 for _, j in combs if j < m])
print(white_cells)

This code attempts to solve the problem by generating combinations of rows and columns but contains some mistakes: it uses incorrect variable names (n, m instead of H, W), misinterprets the selection logic for rows and columns, and does not correctly calculate the number of remaining white cells.

In [ ]:
output = generate_response(df, example.index[0], mode='one-shot')
print(output)

You are a helpful assistant that generates Python code to solve the given problem.
You should respond only with the code, without explanation.

### Problem
You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.

### Solution
N = int(input())
a = list(map(int, input().split()))

ans1 = 0
s = 0
flg = 1
for ai in a:
   

The solution generated by the one-shot was:

In [ ]:
def count_white_cells(h: int, w: int) -> int:

    n = (h + 1) * (w + 1)
    white_count = 0

    for i in range(n):
        row = i // (w + 1)
        col = i % (w + 1)
        if i % 2 == 0:
            white_count += 1 if row % 2 == 1 and col % 2 == 1 else (row % 2 == 0 and col % 2 == 0)
        else:
            white_count += 1 if row % 2 == 0 and col % 2!= 1 else (row % 2!= 0 and col % 2 == 1)

    return white_count

h = int(input())
w = int(input())
h = int(input())
w = int(input())
ans = count_white_cells(h, w)
print(ans)

Which also doesn't solve the problem. And we can see that this one also continues generating more examples and solutions after the one asked.

In [ ]:
output = generate_response(df, example.index[0], mode='few-shot')
print(output)

You are a helpful assistant that generates Python code to solve the given problem.
You should respond only with the code, without explanation.

### Problem
You are given an integer sequence of length N. The i-th term in the sequence is a_i. In one operation, you can select a term and either increment or decrement it by one.

At least how many operations are necessary to satisfy the following conditions?

* For every i (1≤i≤n), the sum of the terms from the 1-st through i-th term is not zero.
* For every i (1≤i≤n-1), the sign of the sum of the terms from the 1-st through i-th term, is different from the sign of the sum of the terms from the 1-st through (i+1)-th term.

Constraints

* 2 ≤ n ≤ 10^5
* |a_i| ≤ 10^9
* Each a_i is an integer.

Input

Input is given from Standard Input in the following format:


n
a_1 a_2 ... a_n


Output

Print the minimum necessary count of operations.

### Solution
N = int(input())
a = list(map(int, input().split()))

ans1 = 0
s = 0
flg = 1
for ai in a:
   

The code generated was:

In [ ]:
n = int(input())
m = int(input())
h = int(input())
w = int(input())
ans = 1

for i in range(h):
    cnt = 0
    for j in range(w):
        if m - h >= i and m - h >= j:
            cnt += 1
        else:
            cnt -= 1
    if cnt == 0 or cnt == 1:
        ans *= 1
    else:
        ans *= -1
ans *= -1
print(ans)

This code is incorrect and unrelated to the problem logic; it uses nested loops and conditions that do not correspond to counting remaining white cells. The approach is confused and does not calculate or print the correct result for the given problem.

The model likely failed to solve the problem in all three approaches because the problem requires a simple, direct mathematical insight, but the prompts or examples may have been unclear or noisy. This can cause the model to overcomplicate or misunderstand the task.

In [ ]:
# Clear GPU cache
torch.cuda.empty_cache()

# Fine-tuning a small LLM for the task

## Fine tuning GPT-2

GPT-2 Medium (https://huggingface.co/openai-community/gpt2-medium) is a transformer-based language model developed by OpenAI, containing approximately 345 million parameters. It is part of the GPT-2 family and was pretrained on a large corpus of internet text using unsupervised learning. Due to its moderate size, GPT-2 Medium is suitable for fine-tuning on domain-specific tasks using limited resources (e.g., Google Colab).

### Imports

We start by doing the necessary library imports

In [ ]:
!pip install transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import drive
import os
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments, Trainer

### Load the model

Before we do the fine-tuning, let's see how the pretrained model does on our task of code generation

In this next cell we load the pretrained model and it's tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 1024)

And now we create the text generation pipeline

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


We will use for demonstration this example prompt that is not in the dataset. The problem is to print YES if the input is even and NO if it is odd.

In [ ]:
prompt = """### Problem:
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight.
Write a Python function that receives an integer `w` (1 <= w <= 100) and returns "YES" if it's possible, or "NO" otherwise.

#### Input:
- An integer w, the weight of the watermelon.

#### Output:
- "YES" if the watermelon can be split into two even positive integers.
- "NO" otherwise.

#### Examples:
Input: 8
Output: YES

Input: 3
Output: NO

Input: 4
Output: YES

### Solution:
"""

With the previous prompt, we generate the model's output

In [ ]:
output = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)
print("Output before fine-tuning:\n")
print(output[0]["generated_text"])

Output before fine-tuning:

### Problem:
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight.
Write a Python function that receives an integer `w` (1 <= w <= 100) and returns "YES" if it's possible, or "NO" otherwise.

### Solution:

Add a new type of function called partition with the function `partition(w,partitionDim,w,partitionDim,w)`. This function takes an integer of `w` and returns a tuple containing two items: `partitionDim` and `partitionDim`.

## Examples:

>>> import multiprocessing import time >>> partition = partition(1, 2, 3, 4) >>> partition(0, 1) 'NO' >>> partition(0, 1) 'YES' >>> partition(1, 10) 'YES' >>> partition(0, 1) 'YES' >>> partition(0, 1) 'YES' >>> partition(0, 1) 'YES' >>> partition(0, 1) 'YES' >>> partition(0, 1) 'YES' >>> partition(0, 1) 'NO' >>> partition(0, 1) 'YES' >>> partition(0, 1) 'NO' >>> partition(0, 1) 'YES' >>> partition(0, 1) 'NO' >>> partition(0, 1) 'NO' >>> partition(0, 1) 'NO' >>> partition(0, 1) 'NO' 

We can see that the model doesn't generate the code for the solution. It tries to start explaining how to solve the problem but does not write any actual code

### Load the dataset

Now we will load out dataset to start the fine-tuning

In [ ]:
drive.mount('/content/drive')

path = 'Colab Notebooks/NLP/NLP_Project'

os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/.shortcut-targets-by-id/17WgJO1gfIBADpYX2jVdb41q7HCbwWcOU/NLP_Project'

In [ ]:
df = pd.read_csv('final_ds.csv')
print(df.head())

                                 problem_description solution_id  \
0  Xenia has a set of weights and pan scales. Eac...         0_0   
1  Xenia has a set of weights and pan scales. Eac...         0_2   
2  Xenia has a set of weights and pan scales. Eac...         0_4   
3  Xenia has a set of weights and pan scales. Eac...         0_6   
4  Xenia has a set of weights and pan scales. Eac...         0_8   

                                       solution_code  \
0  __author__ = 'ratnesh.mishra'\n\nweights = map...   
1  import sys\nsys.setrecursionlimit (1000000)\n\...   
2  import sys\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...   
3  MOD = 10**9 + 7\nI = lambda:list(map(int,input...   
4  to_print = []\ndef dfs(d, ini, s, depth, m):\n...   

               problem_name time_complexity_inferred space_complexity_inferred  
0  339_C. Xenia and Weights                     O(1)                   O(n**2)  
1  339_C. Xenia and Weights                     O(1)                      O(1)  
2  339_C. X

### Fine-tuning with 3k samples

First we will start with a small example, doing the fine-tuning with 3000 samples of the dataset. In this next cell we prepare the text that we will give to the model in the training putting together the descriptions and the code solutions, and splitting it into train and test data.

In [ ]:
df_small = df[["problem_description", "solution_code"]].dropna().sample(3000, random_state=42).reset_index(drop=True)

df_small["text"] = df_small.apply(
    lambda row: f"### Problem:\n{row['problem_description']}\n### Solution:\n{row['solution_code']}", axis=1
)

dataset = Dataset.from_pandas(df_small[["text"]])
dataset = dataset.train_test_split(test_size=0.1)

Now we tokenize the selected data with the model's tokenizer

In [ ]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [ ]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In this next step we create the Trainer, defining the training arguments and the model to be trained. We will train 3 epochs.

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-medium-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)


<ipython-input-13-aa4f4d72a588>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Start training

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.037000
200,1.646100
300,1.539500
400,1.455800
500,1.436700
600,1.412100
700,1.425900
800,1.401400
900,1.293900
1000,1.335100


TrainOutput(global_step=4050, training_loss=1.179877754493996, metrics={'train_runtime': 2023.7052, 'train_samples_per_second': 4.003, 'train_steps_per_second': 2.001, 'total_flos': 7522475625676800.0, 'train_loss': 1.179877754493996, 'epoch': 3.0})

After training we save the model generated

In [ ]:
trainer.save_model("./gpt2-medium-finetuned")
tokenizer.save_pretrained("./gpt2-medium-finetuned")

('./gpt2-medium-finetuned/tokenizer_config.json',
 './gpt2-medium-finetuned/special_tokens_map.json',
 './gpt2-medium-finetuned/vocab.json',
 './gpt2-medium-finetuned/merges.txt',
 './gpt2-medium-finetuned/added_tokens.json',
 './gpt2-medium-finetuned/tokenizer.json')

Now we will generate an output with the same prompt from the beggining to see how the finetuned model behaves

First we create the pipeline with the trained model

In [ ]:
pipe_finetuned = pipeline("text-generation", model="./gpt2-medium-finetuned", tokenizer=tokenizer)

Device set to use cuda:0


And pass the same prompt again

In [ ]:
prompt = """### Problem:
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight.
Write a Python function that receives an integer `w` (1 <= w <= 100) and returns "YES" if it's possible, or "NO" otherwise.

#### Input:
- An integer w, the weight of the watermelon.

#### Output:
- "YES" if the watermelon can be split into two even positive integers.
- "NO" otherwise.

#### Examples:
Input: 8
Output: YES

Input: 3
Output: NO

Input: 4
Output: YES

### Solution:
"""

output_finetuned = pipe_finetuned(prompt, max_new_tokens=256)
print("After fine-tuning:\n", output_finetuned[0]["generated_text"])


After fine-tuning:
 ### Problem:
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight.
Write a Python function that receives an integer `w` (1 <= w <= 100) and returns "YES" if it's possible, or "NO" otherwise.

#### Input:
- An integer w, the weight of the watermelon.

#### Output:
- "YES" if the watermelon can be split into two even positive integers.
- "NO" otherwise.

#### Examples:
Input: 8  
Output: YES

Input: 3  
Output: NO

Input: 4  
Output: YES

### Solution:
w = int(input())
sum = 0
for i in range(1, w+1):
    while(i & 1):
        if(w & (i & 0x00)) == 0:
           sum += 1
    else:
         sum += sum*(i & 0x00)
          sum *= 1
          sum *= i

print('YES' if sum == 'YES' else 'NO')


We can see that the output, even if it's not the correct solution for the problem, is actually well structured python code, an improvement from the first output generated pre-finetuning.

### Fine-tuning with 10k samples

Now we will take a larger piece of the dataset for the finetuning, with 10000 samples, and again divide into train and test sets and tokenize it.

In [ ]:
df_10k = df[["problem_description", "solution_code"]].dropna().sample(10000, random_state=42).reset_index(drop=True)
df_10k["text"] = df_10k.apply(
    lambda row: f"### Problem:\n{row['problem_description']}\n### Solution:\n{row['solution_code']}", axis=1
)

dataset_10k = Dataset.from_pandas(df_10k[["text"]])
dataset_10k = dataset_10k.train_test_split(test_size=0.1)

In [ ]:
tokenized_10k_dataset = dataset_10k.map(tokenize, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

We define the Trainer and the training arguments, this time with 2 epochs.

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-medium-finetuned-10kds",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    learning_rate=5e-5,
    save_steps=5000,
    eval_steps=1000,
    logging_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_10k_dataset["train"],
    eval_dataset=tokenized_10k_dataset["test"],
    tokenizer=tokenizer
)


<ipython-input-10-e0c5a36a666c>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


And start the training

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.961900
200,1.611400
300,1.583300
400,1.469600
500,1.473500
600,1.450400
700,1.364800
800,1.378400
900,1.363800
1000,1.285100


TrainOutput(global_step=9000, training_loss=1.0219143998887803, metrics={'train_runtime': 3736.2941, 'train_samples_per_second': 4.818, 'train_steps_per_second': 2.409, 'total_flos': 1.6716612501504e+16, 'train_loss': 1.0219143998887803, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7891965508460999,
 'eval_runtime': 49.0426,
 'eval_samples_per_second': 20.39,
 'eval_steps_per_second': 10.195,
 'epoch': 2.0}

In [ ]:
trainer.save_model("./gpt2-medium-finetuned-10k")
tokenizer.save_pretrained("./gpt2-medium-finetuned-10k")

('./gpt2-medium-finetuned-10k/tokenizer_config.json',
 './gpt2-medium-finetuned-10k/special_tokens_map.json',
 './gpt2-medium-finetuned-10k/vocab.json',
 './gpt2-medium-finetuned-10k/merges.txt',
 './gpt2-medium-finetuned-10k/added_tokens.json',
 './gpt2-medium-finetuned-10k/tokenizer.json')

Now we will run some metrics on the test set using 30 samples. First we will generate the output with the finetuned version for these samples.

In [ ]:
import torch
from tqdm import tqdm

model_path = "./gpt2-medium-finetuned-10k"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device= -1
)

test_dataset = dataset_10k["test"]

expected_codes = []
generated_codes = []

for example in tqdm(test_dataset.select(range(30))):
    full_text = example["text"]

    try:
        parts = full_text.split("### Problem:\n")[1].split("### Solution:\n")
        description = parts[0].strip()
        real_code = parts[1].strip()
    except (IndexError, AttributeError):
        continue

    prompt = f"### Problem:\n{description}\n### Solution:\n"

    if not prompt.strip():
        continue

    max_prompt_tokens = 768
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_prompt_tokens)
    prompt = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)

    try:
        output = generator(prompt, max_new_tokens=256, num_return_sequences=1, do_sample=False)[0]["generated_text"]
    except RuntimeError as e:
        print("GPU error:", e)
        continue

    generated_code = output.split("### Solution:\n")[-1].strip()

    expected_codes.append(real_code)
    generated_codes.append(generated_code)



Device set to use cpu
100%|██████████| 30/30 [12:44<00:00, 25.49s/it]


In [ ]:
import pandas as pd

df_codes = pd.DataFrame({
    "reference_code": expected_codes,
    "generated_code": generated_codes
})

df_codes.to_csv("codes_comparison.csv", index=False)


Let's also run the model on the previous example to see the output

In [ ]:
prompt = """### Problem:
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight.
Write a Python function that receives an integer `w` (1 <= w <= 100) and returns "YES" if it's possible, or "NO" otherwise.

#### Examples:
Input: 8
Output: YES

Input: 3
Output: NO

### Solution:
"""

output = generator(prompt, max_new_tokens=256, do_sample=True, temperature=0.7)
print(output[0]["generated_text"])


### Problem:
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight.
Write a Python function that receives an integer `w` (1 <= w <= 100) and returns "YES" if it's possible, or "NO" otherwise.

#### Examples:
Input: 8
Output: YES

Input: 3
Output: NO

### Solution:
w=int(input())
for i in range(len(str(w))):
    if str(i)==str(w-1):
        print("YES")
    else:
        print("NO")


We can see that it does not generate the correct code for the problem, but it improves in comparison with the generated output from the very first model, since now we have well structured python code.

We will also get the outputs from the original model for comparison

In [ ]:
from tqdm import tqdm

model_name = "gpt2-medium"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1
)

test_dataset = dataset_10k["test"]

expected_codes_gpt2 = []
generated_codes_gpt2 = []

for example in tqdm(test_dataset.select(range(30))):
    full_text = example["text"]

    try:
        parts = full_text.split("### Problem:\n")[1].split("### Solution:\n")
        description = parts[0].strip()
        real_code = parts[1].strip()
    except (IndexError, AttributeError):
        continue

    prompt = f"### Problem:\n{description}\n### Solution:\n"

    if not prompt.strip():
        continue

    max_prompt_tokens = 768
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_prompt_tokens)
    prompt = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)

    try:
        output = generator(prompt, max_new_tokens=256, num_return_sequences=1, do_sample=False)[0]["generated_text"]
    except RuntimeError as e:
        print("GPU error:", e)
        continue

    generated_code = output.split("### Solution:\n")[-1].strip()

    expected_codes_gpt2.append(real_code)
    generated_codes_gpt2.append(generated_code)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
100%|██████████| 30/30 [28:46<00:00, 57.56s/it]


In [ ]:
df_codes = pd.DataFrame({
    "reference_code": expected_codes_gpt2,
    "generated_code": generated_codes_gpt2
})

df_codes.to_csv("codes_comparison_gpt2.csv", index=False)


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=eef5c3cff69b0071032aab654ac2062804a828d7fa5b82cf4afb07be1ae8d809
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Here's the function to calculate the metrics. Exact Match checks if the generated code is exactly the same as the reference. Levenshtein Similarity measures how similar two strings are based on the number of edit operations needed to match them. BLEU Score evaluates n-gram overlap between generated and reference texts, commonly used in machine translation. ROUGE-L captures the longest common subsequence, focusing on the structural similarity between outputs.

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import numpy as np
import difflib

def exact_match(pred, ref):
    return pred.strip() == ref.strip()

def levenshtein_ratio(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

def bleu_score(pred, ref):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([nltk.word_tokenize(ref)], nltk.word_tokenize(pred), smoothing_function=smoothie)

def rouge_l_score(pred, ref):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    return scorer.score(ref, pred)['rougeL'].fmeasure

def compare_all(expected_codes, generated, label=""):
    print(f"\n=== Metrics for: {label} ===")

    em_list = []
    lev_list = []
    bleu_list = []
    rouge_list = []

    for pred, ref in zip(generated, expected_codes):
        em_list.append(exact_match(pred, ref))
        lev_list.append(levenshtein_ratio(pred, ref))
        bleu_list.append(bleu_score(pred, ref))
        rouge_list.append(rouge_l_score(pred, ref))

    print(f"Exact Match: {np.mean(em_list):.3f}")
    print(f"Levenshtein Similarity: {np.mean(lev_list):.3f}")
    print(f"BLEU Score: {np.mean(bleu_list):.3f}")
    print(f"ROUGE-L Score: {np.mean(rouge_list):.3f}")


In [ ]:
codes_finetuned = pd.read_csv('codes_comparison.csv')
#codes_gpt2 = pd.read_csv('code_comparison_gpt2.csv')

In [ ]:
expected_codes = codes_finetuned["reference_code"].tolist()
generated_codes_finetuned = codes_finetuned["generated_code"].tolist()

#expected_codes_gpt2 = codes_gpt2["reference_code"].tolist()
#generated_codes_gpt2 = codes_gpt2["generated_code"].tolist()

In [ ]:
compare_all(expected_codes_gpt2, generated_codes_gpt2, label="GPT-2 Original")
compare_all(expected_codes, generated_codes_finetuned, label="GPT-2 Fine-tuned")


=== Metrics for: GPT-2 Original ===
Exact Match: 0.000
Levenshtein Similarity: 0.026
BLEU Score: 0.004
ROUGE-L Score: 0.057

=== Metrics for: GPT-2 Fine-tuned ===
Exact Match: 0.000
Levenshtein Similarity: 0.282
BLEU Score: 0.157
ROUGE-L Score: 0.326


The metrics show that the fine-tuned GPT-2 model significantly outperforms the original GPT-2 in generating code closer to the expected solutions. While both models have an Exact Match score of 0, indicating no perfect matches, the fine-tuned model achieves much higher similarity scores across Levenshtein, BLEU, and ROUGE-L metrics. This suggests that fine-tuning helps the model produce code that is structurally and lexically more similar to the reference, improving overall generation quality even if exact reproduction is rare.

## Fine tuning TinyLlama

TinyLlama-1.1B-Chat-v1.0 (https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0) is a compact, chat-optimized language model with 1.1B parameters, based on the Llama 2 architecture. It was pretrained on 3T tokens and fine-tuned using UltraChat and UltraFeedback datasets to improve dialogue quality. Despite its small size, it performs well in chat tasks, especially on limited hardware.

### Imports

We start by doing the necessary library imports

In [ ]:
!pip install -q peft accelerate transformers datasets bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import pandas as pd
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from google.colab import drive
import os
from datasets import Dataset

### Load the model

Before we do the fine-tuning, let's see how the pretrained model does on our task of code generation

In this next cell we will create a text-generation pipeline with the pretrained model

In [ ]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


And we will pass as the messages a problem that is not in our bigger dataset

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful Python assistant."},
    {"role": "user", "content": "Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight. Write a Python function that receives an integer `w` (1 <= w <= 100) and returns 'YES' if it's possible, or 'NO' otherwise."}
]

Now we will generate the output using the messages above as input

In [ ]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You are a helpful Python assistant.</s>
<|user|>
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight. Write a Python function that receives an integer `w` (1 <= w <= 100) and returns 'YES' if it's possible, or 'NO' otherwise.</s>
<|assistant|>
Here's a Python function that returns 'YES' if it's possible to split a watermelon of weight `w` into two parts, each of even positive weight, and 'NO' otherwise:

```python
def is_splitable(w):
    """
    Takes a weight `w` and returns 'YES' if it's possible to split the watermelon into two parts,
    each of even positive weight, or 'NO' otherwise.
    """
    if w <= 0:
        return "Watermelon is not splitable!"
    return "YES" if (w % 2) == 0 else "NO"
```

Here's an example usage:

```python
watermelon_wgt = 100
is_splitable = is_splitable(watermelon_wgt)
print(is_splitable)  # Output: YES
```

This function takes an integer `w` (1 <= w <= 100) as input and returns either 'YES' (if it'

We can see that the model already produces python code for the task, as request in the system prompt. It also tries to explain the code generated.

### Load the dataset

Now we will load out dataset to start the fine-tuning

In [ ]:
drive.mount('/content/drive')

path = 'Colab Notebooks/NLP/NLP_Project'

os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/.shortcut-targets-by-id/17WgJO1gfIBADpYX2jVdb41q7HCbwWcOU/NLP_Project'

In [ ]:
df = pd.read_csv('final_ds.csv')
print(df.head())

                                 problem_description solution_id  \
0  Xenia has a set of weights and pan scales. Eac...         0_0   
1  Xenia has a set of weights and pan scales. Eac...         0_2   
2  Xenia has a set of weights and pan scales. Eac...         0_4   
3  Xenia has a set of weights and pan scales. Eac...         0_6   
4  Xenia has a set of weights and pan scales. Eac...         0_8   

                                       solution_code  \
0  __author__ = 'ratnesh.mishra'\n\nweights = map...   
1  import sys\nsys.setrecursionlimit (1000000)\n\...   
2  import sys\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...   
3  MOD = 10**9 + 7\nI = lambda:list(map(int,input...   
4  to_print = []\ndef dfs(d, ini, s, depth, m):\n...   

               problem_name time_complexity_inferred space_complexity_inferred  
0  339_C. Xenia and Weights                     O(1)                   O(n**2)  
1  339_C. Xenia and Weights                     O(1)                      O(1)  
2  339_C. X

### Fine tuning with 1000 samples

First we will start with a small example, doing the fine-tuning with 1000 samples of the dataset. In this next cell we prepare the text that we will give to the model in the training putting together the descriptions and the code solutions in the messages format, and splitting it into train and test data.

In [ ]:
df_small = df[["problem_description", "solution_code"]].dropna().sample(1000, random_state=42)

In [ ]:
def format_messages(row):
    return [
        {"role": "system", "content": "You are a helpful Python assistant."},
        {"role": "user", "content": row["problem_description"]},
        {"role": "assistant", "content": row["solution_code"]}
    ]

In [ ]:
df_small["messages"] = df_small.apply(format_messages, axis=1)
dataset = Dataset.from_pandas(df_small[["messages"]])
dataset = dataset.train_test_split(test_size=0.1)

Here we define the original model name and tokenizer

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


This next function is used to tokenize the prepared dataset with the model's tokenizer

In [ ]:
def apply_chat_template(example):
    prompt = tokenizer.apply_chat_template(example["messages"], tokenize=False, add_generation_prompt=False)

    tokenized = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


In [ ]:
tokenized_dataset = dataset.map(apply_chat_template)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

This next cell loads the base model in 4-bit precision and prepares it for k-bit training. LoRA adapters are configured to fine-tune only the q_proj and v_proj layers in a memory-efficient way.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In this next step we create the Trainer, defining the training arguments and the model to be trained. We will train 2 epochs.

In [ ]:
training_args = TrainingArguments(
    output_dir="./tinyllama-chat-finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    learning_rate=2e-4,
    logging_steps=100,
    eval_steps=250,
    save_steps=250,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)


<ipython-input-19-d8dc1c25d451>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Start training

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we wi

Step,Training Loss
100,1.537500
200,1.176600
300,1.199600
400,1.178200
500,1.132600
600,1.160100
700,1.147200
800,1.150500
900,1.059700


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=900, training_loss=1.1935513051350912, metrics={'train_runtime': 689.5834, 'train_samples_per_second': 2.61, 'train_steps_per_second': 1.305, 'total_flos': 5726668220006400.0, 'train_loss': 1.1935513051350912, 'epoch': 2.0})

After training we save the model generated

In [ ]:
model.save_pretrained("tinyllama-lora-finetuned")
tokenizer.save_pretrained("tinyllama-lora-finetuned")

('tinyllama-lora-finetuned/tokenizer_config.json',
 'tinyllama-lora-finetuned/special_tokens_map.json',
 'tinyllama-lora-finetuned/tokenizer.model',
 'tinyllama-lora-finetuned/added_tokens.json',
 'tinyllama-lora-finetuned/tokenizer.json')

Now we will generate an output with the same prompt from the beggining to see how the finetuned model behaves

In [ ]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
model = PeftModel.from_pretrained(base_model, "tinyllama-lora-finetuned")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

messages = [
    {"role": "system", "content": "You are a helpful Python assistant."},
    {"role": "user", "content": "Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight. Write a Python function that receives an integer `w` (1 <= w <= 100) and returns 'YES' if it's possible, or 'NO' otherwise."}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
output = pipe(prompt, max_new_tokens=256, do_sample=True)
print(output[0]["generated_text"])

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGen

<|system|>
You are a helpful Python assistant.</s>
<|user|>
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight. Write a Python function that receives an integer `w` (1 <= w <= 100) and returns 'YES' if it's possible, or 'NO' otherwise.</s>
<|assistant|>
w = int(input())
if w % 2 == 0:
    if w == 0:
        print('NO')
        return
    print('YES')
else:
    print('NO')


We can see that the model generated the correct code, but now without the explanations, which is more aligned with the examples from the dataset

### Fine tuning with 5000 samples

Now we will try the finetuning with a little more samples

We take 5000 samples of the dataset, divide again into train and test set, apply the messages format and tokenize it

In [ ]:
df_medium = df[["problem_description", "solution_code"]].dropna().sample(5000, random_state=42)

In [ ]:
df_medium["messages"] = df_medium.apply(format_messages, axis=1)
dataset_medium = Dataset.from_pandas(df_medium[["messages"]])
dataset_medium = dataset_medium.train_test_split(test_size=0.1)

In [ ]:
tokenized_dataset_medium = dataset_medium.map(apply_chat_template)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Here we define again the training arguments and the Trainer, taking the new dataset

In [ ]:
training_args = TrainingArguments(
    output_dir="./tinyllama-chat-finetuned-mediumds",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    learning_rate=2e-4,
    logging_steps=100,
    eval_steps=500,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_medium["train"],
    eval_dataset=tokenized_dataset_medium["test"],
    tokenizer=tokenizer
)


<ipython-input-15-a8e69201f326>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


And start the training

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we wi

Step,Training Loss
100,1.561400
200,1.237800
300,1.226700
400,1.172400
500,1.161700
600,1.138800
700,1.161600
800,1.112300
900,1.087900
1000,1.139900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=4500, training_loss=1.0915350663926866, metrics={'train_runtime': 3303.0539, 'train_samples_per_second': 2.725, 'train_steps_per_second': 1.362, 'total_flos': 2.8633341100032e+16, 'train_loss': 1.0915350663926866, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.0485718250274658,
 'eval_runtime': 58.3072,
 'eval_samples_per_second': 8.575,
 'eval_steps_per_second': 4.288,
 'epoch': 2.0}

Now we will generate again an output for the first prompt to see how the model behaves

In [ ]:
from transformers import BitsAndBytesConfig

base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,
                                bnb_4bit_compute_dtype=torch.bfloat16)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

adapter_path = "./tinyllama-chat-finetuned-mediumds/checkpoint-4500"
model = PeftModel.from_pretrained(base_model, adapter_path)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

messages = [
    {"role": "system", "content": "You are a helpful Python assistant."},
    {"role": "user", "content": "Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight. Write a Python function that receives an integer `w` (1 <= w <= 100) and returns 'YES' if it's possible, or 'NO' otherwise."}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
output = pipe(prompt, max_new_tokens=256, do_sample=True)

print(output[0]["generated_text"])


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

<|system|>
You are a helpful Python assistant.</s>
<|user|>
Petya wants to split a watermelon of weight `w` into two parts, each of even positive weight. Write a Python function that receives an integer `w` (1 <= w <= 100) and returns 'YES' if it's possible, or 'NO' otherwise.</s>
<|assistant|>
def is_positive_even(w):
    if 1000/w == 54:
        return True
    if w % 2 == 0:
        return True
    
    
print("YES", (is_positive_even(w) == True))


Now we will run some metrics on the test set using 20 samples.

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bc8f39e186df29ce86ecfae01a2a7b7de1c6a5340fdcbcaa23126f28de3a6836
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Here's the function to calculate the metrics. Exact Match checks if the generated code is exactly the same as the reference. Levenshtein Similarity measures how similar two strings are based on the number of edit operations needed to match them. BLEU Score evaluates n-gram overlap between generated and reference texts, commonly used in machine translation. ROUGE-L captures the longest common subsequence, focusing on the structural similarity between outputs.

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import numpy as np
import difflib

def exact_match(pred, ref):
    return pred.strip() == ref.strip()

def levenshtein_ratio(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

def bleu_score(pred, ref):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([nltk.word_tokenize(ref)], nltk.word_tokenize(pred), smoothing_function=smoothie)

def rouge_l_score(pred, ref):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    return scorer.score(ref, pred)['rougeL'].fmeasure

def compare_all(expected_codes, generated, label=""):
    print(f"\n=== Metrics for: {label} ===")

    em_list = []
    lev_list = []
    bleu_list = []
    rouge_list = []

    for pred, ref in zip(generated, expected_codes):
        em_list.append(exact_match(pred, ref))
        lev_list.append(levenshtein_ratio(pred, ref))
        bleu_list.append(bleu_score(pred, ref))
        rouge_list.append(rouge_l_score(pred, ref))

    print(f"Exact Match: {np.mean(em_list):.3f}")
    print(f"Levenshtein Similarity: {np.mean(lev_list):.3f}")
    print(f"BLEU Score: {np.mean(bleu_list):.3f}")
    print(f"ROUGE-L Score: {np.mean(rouge_list):.3f}")


Now we will generate the outputs for the original model and for the finetuned model, and run the metrics function on the generated data

In [ ]:
from tqdm import tqdm
from transformers import BitsAndBytesConfig
from peft import PeftModel

def extract_code_from_output(output_text):
    if "<|assistant|>" in output_text:
        return output_text.split("<|assistant|>")[-1].strip()
    else:
        return output_text.strip()

base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,
                               bnb_4bit_compute_dtype=torch.bfloat16)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

adapter_path = "./tinyllama-chat-finetuned-mediumds/checkpoint-4500"
finetuned_model = PeftModel.from_pretrained(base_model, adapter_path)

pipe_orig = pipeline("text-generation", model=base_model, tokenizer=tokenizer, device_map="auto")
pipe_ft = pipeline("text-generation", model=finetuned_model, tokenizer=tokenizer, device_map="auto")

test_dataset = dataset_medium["test"].select(range(20))

expected_codes = []
generated_orig = []
generated_ft = []

for example in tqdm(test_dataset):
    messages = example["messages"]
    description = ""
    real_code = ""

    for m in messages:
        if m["role"] == "user":
            description = m["content"].strip()
        elif m["role"] == "assistant":
            real_code = m["content"].strip()

    if not description or not real_code:
        continue

    prompt_messages = [
        {"role": "system", "content": "You are a helpful Python assistant."},
        {"role": "user", "content": description}
    ]

    prompt = tokenizer.apply_chat_template(prompt_messages, tokenize=False, add_generation_prompt=True)

    output_orig = pipe_orig(prompt, max_new_tokens=256, do_sample=True)[0]["generated_text"]
    code_orig = extract_code_from_output(output_orig)

    output_ft = pipe_ft(prompt, max_new_tokens=256, do_sample=True)[0]["generated_text"]
    code_ft = extract_code_from_output(output_ft)

    expected_codes.append(real_code)
    generated_orig.append(code_orig)
    generated_ft.append(code_ft)

df_orig = pd.DataFrame({
    "reference_code": expected_codes,
    "generated_code_original": generated_orig
})
df_orig.to_csv("codes_comparison_tinyllama_original.csv", index=False)

df_ft = pd.DataFrame({
    "reference_code": expected_codes,
    "generated_code_finetuned": generated_ft
})
df_ft.to_csv("codes_comparison_tinyllama_finetuned.csv", index=False)

compare_all(expected_codes, generated_orig, label="TinyLlama Original")
compare_all(expected_codes, generated_ft, label="TinyLlama Fine-tuned")


Device set to use cuda:0
Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCo


=== Metrics for: TinyLlama Original ===
Exact Match: 0.000
Levenshtein Similarity: 0.178
BLEU Score: 0.105
ROUGE-L Score: 0.239

=== Metrics for: TinyLlama Fine-tuned ===
Exact Match: 0.000
Levenshtein Similarity: 0.202
BLEU Score: 0.148
ROUGE-L Score: 0.300


The evaluation metrics show that the fine-tuned TinyLlama model outperforms the original in all measured aspects: Levenshtein similarity increased from 0.178 to 0.202, BLEU score improved from 0.105 to 0.148, and ROUGE-L rose from 0.239 to 0.300. Although these improvements are moderate, they reflect consistent gains in code generation quality. The relatively small increase is expected because the base model was already capable of generating Python code to some extent before fine-tuning.